## 1 - SETUP

In [1]:
####################################################################################################
####################################################################################################
# 1 - SETUP
# !pip install --upgrade --user XlsxWriter
# !pip install --upgrade --user ortools
# !pip install --upgrade --user pandas
# !pip install --upgrade --user numpy
# !pip install --upgrade --user tkinter
# !pip install --upgrade --user swifter
# conda install -c plotly plotly-orca psutil requests
# pip install pandarallel --upgrade --user
# pip install pixiedust

# Pandarallel progress bars
# !pip install ipywidgets  --upgrade --user
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

# Multiprocessing
import multiprocessing
import psutil # To get number of cores

# PANDAS, NUMPY
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import swifter # Used for optmize pandas apply

pd.set_option('display.max_rows', 200) # Display maximum

import xlsxwriter
import numpy as np

# Itertools
from  more_itertools import unique_everseen

# Random
from random import *
import random as random
from random import sample

# Dask
import dask.array as da
import dask.dataframe as dd

# Garbage collector
import gc

# TKINTER
from tkinter import*
from tkinter import filedialog

# Date/time for naming export files
import time
from time import process_time 

# OR-TOOLS
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

# CPLEX
import sys

import docplex.mp
from docplex.mp.model import Model

import docplex.mp.sdetails 
from docplex.mp.environment import Environment

env = Environment()
# env.print_information()

# PLOTLY
from plotly.offline import plot
import plotly.figure_factory as ff
import os # Used to save PLOTLY as image

# Debugger
# import pixiedust # %%pixie_debugger on top off a cell

####################################################################################################
####################################################################################################
# 2 - IMPORT INPUT
# Import Excel file with TK dialog
root = Tk() # Create TK root
root.withdraw() # Hide the main window
root.call('wm', 'attributes', '.', '-topmost', True)
inputfile = filedialog.askopenfilename(multiple=True)
%gui tk
print("Imported file: ", inputfile)

Imported file:  ('C:/Users/alexa/UG_SCHED/Inputs/2019_12_07_INPUT_MIP_MODEL_2044_TASKS.xlsx',)


## 2 - PREPARE INPUT

In [2]:
####################################################################################################
####################################################################################################
# 1 - Transfer input file to Pandas DF
input_machines = pd.read_excel(inputfile[0], sheet_name='MACHINES', usecols = "A:C")
input_tasks = pd.read_excel(inputfile[0], sheet_name='JOBS')

####################################################################################################
####################################################################################################
# 2 - Replace PD and SD by 0, NA by 1 in input_tasks
machines = {'PD': 0,'SD': 0, np.NaN:1} # Define dictionary 
input_tasks['DEV TYPE'] = input_tasks['DEV TYPE'].map(machines)

####################################################################################################
####################################################################################################
# 3 - Replace hex IDs by integer IDs
# Adds ID_integer (row index) to input_tasks
# "Try" allow cells to be run many times.
try:
    input_tasks.insert(0, 'IDNUM', range(0, len(input_tasks)))
except ValueError: # Error handling in case of floating value
    pass # do nothing

# Create dictionary: ID hex to ID int
id_dic = dict(zip(input_tasks['ID'], input_tasks['IDNUM'])) # Dictionary ID hex --> ID integer

####################################################################################################
####################################################################################################
# 4 - Split "Predecessor IDs" in different coluns and add them to input_tasks
split_pred = input_tasks['Predecessor IDs'].str.split(';', n=-1, expand=True) # ";" is the separator in input file
tasks_concat = pd.concat([input_tasks,split_pred], axis=1)

for col in tasks_concat.columns[6:]:
    tasks_concat[col] = tasks_concat[col].map(id_dic) # Maps ID int to previous ID hex 
    tasks_concat.rename(columns={col:'Pred_'+ str(col)}, inplace=True) # Rename columns
    
del tasks_concat['ID']
del tasks_concat['Predecessor IDs'] 

####################################################################################################
####################################################################################################
# 5 - Create Profit and Precedessors DF
# Profit
tasks_profit = tasks_concat.copy()
for col in tasks_profit.columns[1:]:
    if col == "PROFIT":
        continue
    del tasks_profit[col]
tasks_profit = pd.DataFrame(tasks_profit['PROFIT']).astype(int) # Change to int values for CP-SAT
tasks_profit_matrix = pd.concat([tasks_profit]*len(input_machines.index), ignore_index=True, axis=1)

# Predecessors
tasks_pred = tasks_concat.copy()
for col in tasks_pred.columns[0:4]:  
    if col == "DEV TYPE":
        continue
    del tasks_pred[col]

# Convert to int
tasks_pred.fillna(-1,inplace=True) # Replace NAN by -1
tasks_pred = tasks_pred.applymap(str) # Convert object to str
tasks_pred = tasks_pred.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

####################################################################################################
####################################################################################################
# 6 - Split predecessors in Hard and Soft predecessors
physical_access = tasks_concat[tasks_concat['DEV TYPE'] == 0]["IDNUM"] # If DEV TYPE == 0, task may be a Hard Predecessor

# Hard Predecessors
tasks_pred_hard = tasks_pred.applymap(lambda x: x if x in physical_access else -1) # Lambda to consider hard constraints
tasks_pred_hard.insert(0, 'IDNUM', range(0, len(input_tasks))) # Insert IDNUM again
del tasks_pred_hard["DEV TYPE"] # Delete DEV TYPE
tasks_pred_hard.fillna(-1,inplace=True) # Replace NAN by -1
tasks_pred_hard = tasks_pred_hard.applymap(str) # Convert object to str
tasks_pred_hard = tasks_pred_hard.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1)  # Convert str to integer

# Soft Predecessors
tasks_pred_soft = tasks_pred.applymap(lambda x: x if x not in physical_access else -1) # Lambda to disconsider hard constraints
tasks_pred_soft.insert(0, 'IDNUM', range(0, len(input_tasks))) # Insert IDNUM again
del tasks_pred_soft["DEV TYPE"] # Delete DEV TYPE
tasks_pred_soft.fillna(-1,inplace=True) # Replace NAN by -1
tasks_pred_soft = tasks_pred_soft.applymap(str) # Convert object to str
tasks_pred_soft = tasks_pred_soft.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1)  # Convert str to integer

####################################################################################################
####################################################################################################
# 07 - Tasks successors
# Create temporary tasks_succ DF
tasks_succ = tasks_concat.copy()
tasks_succ.drop(tasks_succ.columns[[1, 2, 3]], axis=1, inplace = True) # Delete DEV TYPE, Driving quantity, and PROFIT columns

# Stack predecessors in a list
pred_list = tasks_succ[tasks_succ.columns[1]] # Create list with 1st column of predecessors
for col in tasks_succ.columns[2:]:
    pred_list = pred_list.append(tasks_succ[col], ignore_index=True) # Stack other columns of predecessors in a list

# Stack IDNUM in a list
index_col = len(tasks_succ.columns)-1 # Number of predecessors columns to repeat IDNUM in a list
succ_id = tasks_succ['IDNUM'] # Create list of IDNUM
for col in range(index_col-1): # -1 since 1st was already created
    succ_id = succ_id.append(tasks_succ['IDNUM'], ignore_index=True) # Stack other columns of predecessors in a list

# Combine predecessors list and IDNUM repeated list in a DF
tasks_succ = pd.concat([succ_id,pred_list], axis=1)
tasks_succ.columns = ['SUCC', 'IDNUM'] # Rename the two columns

# Group lists in multiple columns
tasks_succ = tasks_succ.groupby(['IDNUM'])['SUCC'].apply(lambda x: ';'.join(x.astype(str))).reset_index() # Group by predecessos (IDNUM) and concatenate
tasks_succ = pd.merge(tasks_succ,tasks_concat[['IDNUM']], how='outer') # Complete tasks_succ with IDNUM that do not have successors
tasks_succ.sort_values(['IDNUM'], inplace = True) # Sort by IDNUM
tasks_succ = tasks_succ.reset_index(drop=True) # Reset index
tasks_succ_split = tasks_succ['SUCC'].str.split(';', n=-1, expand=True) # Split sucessors in columns
tasks_succ = pd.concat([tasks_succ,tasks_succ_split], axis=1) # Add split sucessors to DF
for col in tasks_succ.columns[2:]: # Rename sucessors columns 
    tasks_succ.rename(columns={col:'Succ_'+ str(col)}, inplace=True)
del tasks_succ['SUCC'] # Delete column with grouped sucessors

# Convert to int
tasks_succ.fillna(-1,inplace=True) # Replace NAN by -1
tasks_succ = tasks_succ.applymap(str) # Convert object to str
tasks_succ = tasks_succ.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

####################################################################################################
####################################################################################################
# 8 - Create pij duration matrix
pij_duration = tasks_concat[["IDNUM","DEV TYPE","Driving quantity"]].copy()
for i in range(len(input_machines)): # Loop through each machine and create a column for each
    pij_duration['Mach_'+str(i)] = -(-pij_duration['Driving quantity']//input_machines.iloc[i, 2]*(pij_duration['DEV TYPE'] == input_machines.iloc[i, 0])) # If machine does not process task type, duration will be 0. Double slash // for rounding
pij_duration.drop(pij_duration.columns[[0, 1, 2]], axis=1, inplace = True) # Delete IDNUM, DEV TYPE, and Driving quantity columns
pij_duration = pij_duration.astype(int) # Change to int to work in CP-SAT

####################################################################################################
####################################################################################################
# 9 - l-time parameter (upper-bound for time index) - Calculation with greedy allocation
# Create a DF to store cumulative duration for tasks that have predecessors
tasks_cum_duration = tasks_concat.copy()
tasks_cum_duration.drop(tasks_cum_duration.columns[1:], axis=1, inplace = True) # Keep only IDNUM column
tasks_cum_duration['Cumulative duration'] = 0 # Create column to store cumulative duration for tasks that have predecessors

# Initialize group of avaiable tasks (tasks without predecessors)
tasks_available = tasks_pred.copy() # This is used for the whole loop, being updated with -1 as tasks are allocated
tasks_available.drop(tasks_available.columns[0], axis=1, inplace = True) # Keep only 'Pred_' columns
max_num_pred = len(tasks_available.columns) # Maximum number of predecessors that can happen
tasks_available_list = tasks_pred.copy() # This will be a list
tasks_available_list['Count_empty'] = tasks_available.isin([-1]).sum(axis=1) # Column to count empty predecessors
tasks_available_list = tasks_available_list.loc[tasks_available_list['Count_empty'] == max_num_pred] # Filter tasks that are available (do not have predecessors)
tasks_available_list = tasks_available_list.index.values.astype(int).tolist() # list with available tasks to be allocated

# Initialize machines
machine_time = input_machines.copy()
machine_time['Available at time...'] = 0 # Create column to store time of availability of each machine

# Create a DF to store allocation
processed_tasks_allocation = pd.DataFrame(columns=['IDNUM','Machine', 'Task start (xi)', 'Duration'])

# Allocate each and update available tasks
i=0 # Used for processed_tasks_allocation and for printing loop counter
while len(tasks_available_list) != 0: # Allocate loop while available tasks exists
# Allocate first task of tasks_available_list
    process_task = tasks_available_list[0] # Select task to be processed (first)
    process_task_mach_type = tasks_concat.iloc[process_task,1] # Get machine TYPE

    process_task_allocated_mach = machine_time.loc[machine_time['TYPE'] == process_task_mach_type] # Select machine to allocate --> filter by TYPE
    process_task_allocated_mach_min_time = process_task_allocated_mach['Available at time...'].min() # Select machine to allocate --> get the one with minimum available time
#     process_task_allocated_mach = process_task_allocated_mach.loc[process_task_allocated_mach['Available at time...'] == process_task_allocated_mach_min_time].index[0] # If more than one under criteria, get smallest ID --> If this is online, comment next 4 lines
    process_task_allocated_mach = process_task_allocated_mach.loc[process_task_allocated_mach['Available at time...'] == process_task_allocated_mach_min_time] # Get the machines with minimum available time
    process_task_allocated_mach_max_rate = process_task_allocated_mach['RATE'].max() # Maximum rate of available machines
    process_task_allocated_mach = process_task_allocated_mach.loc[process_task_allocated_mach['RATE'] == process_task_allocated_mach_max_rate].index[0] # From the one above, get the highest rate machines. If more than one available get the minimum ID
    
    process_task_dr_quantity = tasks_concat.iloc[process_task,2] # Get driving quantity
    process_task_duration = process_task_dr_quantity / input_machines.iloc[process_task_allocated_mach,2] # Get duration based on machine rate
    if process_task_duration != int(process_task_duration): # Check if process_task_duration is integer --> It has to be for MIP solvers
        process_task_duration = int(process_task_duration) + 1 # Round up if not integer
    process_task_cum_duration = process_task_duration + tasks_cum_duration.iloc[process_task,1] # Update cumulative duration of task predecessors with duration of task
        
    machine_time.iloc[process_task_allocated_mach,3] += process_task_duration # Update machine time with processed task duration
    machine_time.iloc[process_task_allocated_mach,3] = max(machine_time.iloc[process_task_allocated_mach,3], process_task_cum_duration) # Update machine time with maximum of (updated machine time, cumulative duration)

# Remove processed task from tasks_available_list
    tasks_available = tasks_available.drop(process_task) # Drop the row of the processed task
    tasks_available_list.remove(process_task) # Remove processed task from available list
    processed_tasks_allocation.loc[i] = process_task # Updated list of processed tasks --> IDNUM. LOC to include row
    processed_tasks_allocation.iloc[i,1] = process_task_allocated_mach # Updated list of processed tasks --> Allocated machine
    processed_tasks_allocation.iloc[i,2] = machine_time.iloc[process_task_allocated_mach,3] - process_task_duration # Updated list of processed tasks --> Task start (xi)
    processed_tasks_allocation.iloc[i,3] = process_task_duration # Updated list of processed tasks --> Duration

# Update processed task time (end) to 'Cumulative duration' of its sucessors
    process_task_succ = tasks_succ.loc[tasks_succ['IDNUM'] == process_task,tasks_succ.columns[1:]].values # Get successors of processed task
    process_task_succ = process_task_succ[process_task_succ!=-1].tolist() # Ignore -1 , since it represents NAN    
    if len(process_task_succ)> 0:
        for succ in process_task_succ:
            tasks_cum_duration.iloc[succ,1] = machine_time.iloc[process_task_allocated_mach,3] # Get value of finish of precessed task
# Replace processed task by -1 in tasks_available 'Pred_' coluns
    tasks_available = tasks_available.replace(process_task, -1)
    
# Check available tasks tasks (without predecessors)
    tmp_tasks_available_list = tasks_available.copy() # This one is temporary
    tmp_tasks_available_list['Count_empty'] = tasks_available.isin([-1]).sum(axis=1) # Column to count empty predecessors
    tmp_tasks_available_list = tmp_tasks_available_list.loc[tmp_tasks_available_list['Count_empty'] == max_num_pred] # Filter tasks that are available (do not have predecessors)
    tmp_tasks_available_list = tmp_tasks_available_list.index.values.astype(int).tolist() # list with available tasks to be allocated

    for task in tmp_tasks_available_list: # Update tasks_available_list with new available tasks but keeping the order
        if task not in tasks_available_list:
            tasks_available_list.append(task)

# Print allocation for each loop
    i+=1
#     print('----- Loop:',i)
#     print('Processed task:',process_task,'  - Machine:',process_task_allocated_mach,'  - Duration:',process_task_duration,'  - Available tasks:',tasks_available_list)
#     print('\n',machine_time,'\n')
#     print('\n',tasks_cum_duration,'\n')

processed_tasks_allocation.sort_values(['IDNUM'], inplace = True) # Sort by IDNUM
l = machine_time['Available at time...'].max() # Upper-bound for time index
if l != int(l): # Check if L is integer --> It has to be for MIP solvers
    l = int(l) + 1 # Round up if not integer
else:
    l = int(l)
    
####################################################################################################
####################################################################################################
# 10 - Create Qi values
# Hard
Qi = tasks_pred_hard.copy() # DF that copies all tasks_pred_hard DF
Qi['-1 type'] = tasks_pred_hard.eq(-1).sum(axis=1) # Aux column with number of ignored predecessors (-1) values
Qi['Qi'] = len(tasks_pred_hard.columns) - Qi['-1 type'] - 1 # Number of hard predecessors = number of columns - ignored -1 predecessors - 1 (IDNUM)
Qi_hard = Qi[['IDNUM','Qi']]

# Soft
Qi = tasks_pred_soft.copy() # DF that copies all tasks_pred_soft DF
Qi['-1 type'] = tasks_pred_soft.eq(-1).sum(axis=1) # Aux column with number of ignored predecessors (-1) values
Qi['Qi'] = len(tasks_pred_soft.columns) - Qi['-1 type'] - 1 # Number of hard predecessors = number of columns - ignored -1 predecessors - 1 (IDNUM)
Qi_soft = Qi[['IDNUM','Qi']]

# Hard and Soft
Qi = pd.DataFrame({'IDNUM':Qi_hard['IDNUM'], 'Qi':(Qi_hard['Qi'] + Qi_soft['Qi'])})

####################################################################################################
####################################################################################################
# 11 - Allowed machines for each task
allowed_machines = pd.merge(tasks_concat, input_machines, left_on=['DEV TYPE'], right_on=['TYPE'])
allowed_machines = allowed_machines.groupby('IDNUM').aggregate(lambda x: x.unique().tolist())
allowed_machines = allowed_machines[['MACHINE']]
allowed_machines

####################################################################################################
####################################################################################################
# 12 - Get discount rate, foundation & reserves timing costs and period costs
input_parameters = pd.read_excel(inputfile[0], sheet_name='PARAMETERS', usecols = "A:B")
day_disc_rate = input_parameters.iloc[1,1] # discount rate per day
costs_found_res_timing = input_parameters.iloc[2,1] # US$ per whole project
costs_period = int(input_parameters.iloc[3,1]) # US$/d

####################################################################################################
####################################################################################################
#13 - Tasks profit (i) vs j vs t
tasks_profit_xit = pd.DataFrame(np.tile(tasks_profit.to_numpy(),int(l))) # DF with profit repeated l columns
discount_factors_xit = np.fromfunction(lambda i, j: 1/(1+day_disc_rate)**j, (len(input_tasks), int(l))) # Discount rates multiplying factors for each
discount_factors_xit = pd.DataFrame(discount_factors_xit) # DF with previous line data
tasks_profit_xijt = (tasks_profit_xit*discount_factors_xit).astype(int) # Multiply profit and discount factors
tasks_profit_xijt['0'] = tasks_profit_xijt.values.tolist() # Group columns into lists into one cell
tasks_profit_xijt.drop(tasks_profit_xijt.columns.difference(['0']), 1, inplace=True) # Drop all columns but the list one
tasks_profit_xijt = pd.DataFrame(np.tile(tasks_profit_xijt.to_numpy(),len(input_machines))) # Repeat the list column (with discounted profit) j-machine times
tasks_profit_xijt = tasks_profit_xijt.values.tolist() # Get list from df

####################################################################################################
####################################################################################################
# 14 - Inputs avaiable
# input_tasks
# input_machines
# allowed_machines
# tasks_concat
# tasks_profit
# tasks_profit_matrix
# tasks_profit_xijt
# tasks_pred
# tasks_pred_hard
# tasks_pred_soft
# tasks_succ
# pij_duration
# processed_tasks_allocation # For L calculation
# l
# Qi_hard
# Qi_soft
# Qi
# day_disc_rate
# costs_found_res_timing # Does not affect objective function
# costs_period

####################################################################################################
####################################################################################################
# 15 - Garbage collector --> Clear RAM
# %whos # Get objects created
# %whos DataFrame # DataFrames created
# %whos list # Lists created
del [[process_task_succ,
    tasks_available_list,
    tmp_tasks_available_list,
    discount_factors_xit,
    input_parameters,
    machine_time,
    split_pred,
    tasks_available,
    tasks_cum_duration,
    tasks_profit_xit,
    tasks_succ_split]] # Del DFs and lists not in use

gc_extra_call = gc.collect() # Garbage collector extra call to save RAM

## 3 - CBC - xi optimizing makespan (l upper-bound alternative)

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1
    
####################################################################################################
####################################################################################################
# 2 - Create matrices to store variables
cmax = 0 # Makespan

xi = [] # Task start
for i in range(len(input_tasks)): # len(input_tasks) = number of tasks
    xi.append(0)
    
yij = [] # 1 if machine j process task i. Otherwise, 0
for i in range(len(input_tasks)): # len(input_tasks) = number of tasks
    dim1 = []
    for j in range(len(input_machines)): # len(input_machines) = number of machines
        dim1.append(0)
    yij.append(dim1)

wik = [] # 1 if task i happens before task k. Otherwise, 0
for i in range(len(input_tasks)):
    dim1 = []
    for k in range(len(input_tasks)):
        dim1.append(0)
    wik.append(dim1)

####################################################################################################
####################################################################################################
# 3 - Create matrices to store constraints
const1i=[]
for i in range(len(input_tasks)):
    const1i.append(0)

const2i=[]
for i in range(len(input_tasks)):
    const2i.append(0)
    
const3ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(input_tasks)):
        dim1.append(0)
    const3ik.append(dim1)

const4ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(input_tasks)):
        dim1.append(0)
    const4ik.append(dim1)
    
const5ikj=[]
for i in range(len(input_tasks)):
    dim2 = []
    for k in range(len(input_tasks)):
        dim1 = []
        for j in range(len(input_machines)):
            dim1.append(0)
        dim2.append(dim1)
    const5ikj.append(dim2)

const6ikjh=[]
for i in range(len(input_tasks)):
    dim3 = []
    for k in range(len(input_tasks)):
        dim2 = []
        for j in range(len(input_machines)):
            dim1 = []
            for h in range(len(input_machines)):
                dim1.append(0)
            dim2.append(dim1)
        dim3.append(dim2)
    const6ikjh.append(dim3)

const7i=[]
for i in range(len(input_tasks)):
    const7i.append(0)

####################################################################################################
####################################################################################################
# 4 - MIP Model
# Objective function and variable
solver = pywraplp.Solver('UG scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective = solver.Objective()

cmax = solver.IntVar(0, l, 'cmax') # Consider L already as an upper bound to makespan
for i in range(len(input_tasks)):
    xi[i] = solver.IntVar(0, l, 'xi'+str(i)) # L as an upper bound
    for j in range(len(input_machines)):
        yij[i][j] = solver.BoolVar('yij'+str(i)+'-'+str(j))
    for k in range(len(input_tasks)):
        wik[i][k] = solver.BoolVar('wik'+str(i)+'-'+str(k))            
objective.SetCoefficient(cmax, 1) # Objective function = Cmax

objective.SetMinimization()

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0

print("CONST 1") # sum(yij) = 1 if j in allowed_machines --> ALL tasks will be allocated
for i in range(len(input_tasks)):
    num_const1 += 1
    const1i[i] = solver.Constraint(1, 1)
    for j in range(len(input_machines)):
        if j in allowed_machines.iloc[i,0]: # j in allowed machines
            const1i[i].SetCoefficient(yij[i][j], 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(yij) = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    const2i[i] = solver.Constraint(0, 0)
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]: # j NOT in allowed machines
            const2i[i].SetCoefficient(yij[i][j], 1)
print(num_const2, "constraints \n")

print("CONST 3")  # Precedencies (hard + soft) --> xi >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const3 += 1
                const3ik[i][k] = solver.Constraint(0, solver.infinity()) # >= 0
                const3ik[i][k].SetCoefficient(xi[i], 1) # xi
                const3ik[i][k].SetCoefficient(xi[k], -1) # -xk
                for j in range(len(input_machines)):
                    const3ik[i][k].SetCoefficient(yij[k][j], - int(pij_duration.iloc[k,j])) # - sum(pkj*ykj) for all j
print(num_const3, "constraints \n")

print("CONST 4")  # No-overlapping 1 -->  For 2 tasks i and k: i happens before k OR k before i --> xi + sum(pij*yij) - xk - l*(1-wik) <= 0 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const4 += 1
            const4ik[i][k] = solver.Constraint(-solver.infinity(), l) # <= L
            const4ik[i][k].SetCoefficient(xi[i], 1) # xi            
            const4ik[i][k].SetCoefficient(xi[k], -1) # -xk
            for j in range(len(input_machines)):
                    const4ik[i][k].SetCoefficient(yij[i][j], int(pij_duration.iloc[i,j])) # + sum(pij*yij) for all j
            const4ik[i][k].SetCoefficient(wik[i][k], l) # L*wik
print(num_const4, "constraints \n")

print("CONST 5")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const5 += 1
                const5ikj[i][k][j] = solver.Constraint(-solver.infinity(), 1) # <= 1
                const5ikj[i][k][j].SetCoefficient(yij[i][j], 1) # yij
                const5ikj[i][k][j].SetCoefficient(yij[k][j], 1) # ykj
                const5ikj[i][k][j].SetCoefficient(wik[i][k], -1) # -wik
                const5ikj[i][k][j].SetCoefficient(wik[k][i], -1) # -wki
print(num_const5, "constraints \n")

print("CONST 6")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const6 += 1
                        const6ikjh[i][k][j][h] = solver.Constraint(-solver.infinity(), 2) # <= 2
                        const6ikjh[i][k][j][h].SetCoefficient(yij[i][j], 1) # yij
                        const6ikjh[i][k][j][h].SetCoefficient(yij[k][h], 1) # ykh
                        const6ikjh[i][k][j][h].SetCoefficient(wik[i][k], 1) # wik
                        const6ikjh[i][k][j][h].SetCoefficient(wik[k][i], 1) # wki
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const7 += 1
    const7i[i] = solver.Constraint(0, solver.infinity()) # >= 0
    const7i[i].SetCoefficient(cmax, 1) # cmax
    const7i[i].SetCoefficient(xi[i], -1) # -xi
    for j in range(len(input_machines)):
        const7i[i].SetCoefficient(yij[i][j], - int(pij_duration.iloc[i,j])) # - sum(pij*yij) for all j
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
parameters = pywraplp.MPSolverParameters()

if time_limit > 0:
    solver.SetTimeLimit(time_limit) # Set time limit

if maximum_gap_allowed > 0:
    parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP

solver.SetNumThreads(num_threads) # Use multi threading / cores
status = solver.Solve(parameters)

####################################################################################################
####################################################################################################
# 5 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if yij[i][j].solution_value() == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = xi[i].solution_value() # Start
            mip_allocation_output[row_num][2] = xi[i].solution_value() + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in (solver.OPTIMAL, solver.FEASIBLE):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if yij[i][j].solution_value() == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**xi[i].solution_value())
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(objective.BestBound() - solver.Objective().Value())/solver.Objective().Value() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CBC" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_makespan" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = solver.NumVariables()
mip_stats_output[0][7] = solver.NumConstraints()
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/1000/3600 # Wall time in hours
mip_stats_output[0][10] = solver.Objective().Value() # Optimized value
mip_stats_output[0][11] = objective.BestBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 6 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 7 - Export linear model as txt
model = solver.ExportModelAsLpFormat(False)
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 8 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == solver.OPTIMAL:  # Optimal solution found 
    print("Optimized value: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
else:  # Optimal solution not found
    if status == solver.FEASIBLE:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in (solver.OPTIMAL, solver.FEASIBLE):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(objective.BestBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % solver.NumVariables())
    print("Total number of constraints: %i" % solver.NumConstraints())
    print("Total number of branch-and-bound nodes: %i \n" % solver.nodes())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % (solver.WallTime()/1000))
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    print("Gap: %.2f" % result_gap)   

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 9 - Update L value
l = int(result_makespan)

## 4 - CBC - xijt (time-indexed)

In [ ]:
####################################################################################################
####################################################################################################
# 13 - Tasks profit (i) vs j vs t (same from ) --> from PREPARE INPU --> Rerun required if running code below multiple times (updating L-value)
tasks_profit_xit = pd.DataFrame(np.tile(tasks_profit.to_numpy(),int(l))) # DF with profit repeated l columns
discount_factors_xit = np.fromfunction(lambda i, j: 1/(1+day_disc_rate)**j, (len(input_tasks), int(l))) # Discount rates multiplying factors for each
discount_factors_xit = pd.DataFrame(discount_factors_xit) # DF with previous line data
tasks_profit_xijt = (tasks_profit_xit*discount_factors_xit).astype(int) # Multiply profit and discount factors
tasks_profit_xijt['0'] = tasks_profit_xijt.values.tolist() # Group columns into lists into one cell
tasks_profit_xijt.drop(tasks_profit_xijt.columns.difference(['0']), 1, inplace=True) # Drop all columns but the list one
tasks_profit_xijt = pd.DataFrame(np.tile(tasks_profit_xijt.to_numpy(),len(input_machines))) # Repeat the list column (with discounted profit) j-machine times
tasks_profit_xijt = tasks_profit_xijt.values.tolist() # Get list from df

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value considered for optimization: %i \n" % l)
l_input = l

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - Create matrices to store variables
cmax = 0 # Makespan will be used to account for period costs

xijt = []
for i in range(len(input_tasks)): # len(input_tasks) = number of tasks
    dim2 = []
    for j in range(len(input_machines)): # len(input_machines) = number of machines
        dim1 = []
        for t in range(int(l)): # l = maximum duration of all tasks (project timeframe)
            dim1.append(0)
        dim2.append(dim1)
    xijt.append(dim2)
    
wi = [0]*len(input_tasks)

####################################################################################################
####################################################################################################
# 3 - Create matrices to store constraints
const1i=[]
for i in range(len(input_tasks)):
    const1i.append(0)
    
const2tj=[]
for t in range(int(l)):
    dim1=[]
    for j in range(len(input_machines)):
        dim1.append(0)
    const2tj.append(dim1)
    
const3i=[]
for i in range(len(input_tasks)):
    const3i.append(0)

const4ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(tasks_pred.columns)-1): # Maximum number of predecessors that can happen
        dim1.append(0)
    const4ik.append(dim1)

const5ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(tasks_pred.columns)-1): # Maximum number of predecessors that can happen
        dim1.append(0)
    const5ik.append(dim1)

const6i=[]
for i in range(len(input_tasks)):
    const6i.append(0)
    
const7i=[]
for i in range(len(input_tasks)):
    const7i.append(0)
    
####################################################################################################
####################################################################################################
# 4 - MIP Model
# Objective function and variable
solver = pywraplp.Solver('UG scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective = solver.Objective()

cmax = solver.IntVar(0, l, 'cmax') # Makespan used to subtract period costs
for i in range(len(input_tasks)):
    wi[i] = solver.BoolVar('wi'+str(i))
    for j in range(len(input_machines)):
        for t in range(int(l)):
            xijt[i][j][t] = solver.BoolVar('xijt'+str(i)+'-'+str(j)+'-'+str(t))
            objective.SetCoefficient(xijt[i][j][t], tasks_profit.iloc[i,0]/((1+day_disc_rate)**t)) # daily discount rate used
objective.SetCoefficient(cmax, -costs_period) # Objective function: - Cmax*costs_period (US$/d)
objective.SetMaximization()
 
# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0

print("CONST 1") # wi + sum(xijt) = 1 --> wi = 1 => tasks not processed. Otherwise: wi = 0
for i in range(len(input_tasks)):
    num_const1 += 1
    const1i[i] = solver.Constraint(1, 1)
    const1i[i].SetCoefficient(wi[i], 1)
    for j in range(len(input_machines)):
        for t in range(int(l)):
            const1i[i].SetCoefficient(xijt[i][j][t], 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(xijs) <= 1 --> For each machine, in each time t only one task can occupy the machine
for t in range(int(l)):
    for j in range(len(input_machines)):
        num_const2 += 1
        const2tj[t][j] = solver.Constraint(-solver.infinity(), 1)
        for i in range(len(input_tasks)):
            for s in range(max(int(t-pij_duration.iloc[i,j]+1),0),t+1): # s is the time window that includes processing time (duration)
                const2tj[t][j].SetCoefficient(xijt[i][j][s], 1)
print(num_const2, "constraints \n")
                    
print("CONST 3") # Hard precedencies --> Qi_hard*sum(xijt) <= sum(xkjt) --> i only happens if ALL k hard predecessors happens  All hard pred as requirement for xijt = 1. Otherwise, xijt = 0
for i in range(len(input_tasks)):
    const3i[i] = solver.Constraint(-solver.infinity(), 0)
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        num_const3 += 1
        for j in range(len(input_machines)):
            for t in range(int(l)):
                const3i[i].SetCoefficient(xijt[i][j][t], int(Qi_hard.iloc[i,1])) # Qi_hard*sum(xijt) for all j and t
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                for j in range(len(input_machines)):
                    for t in range(int(l)):
                        const3i[i].SetCoefficient(xijt[k][j][t], -1) # - sum(xkjt) for all t, j and k
print(num_const3, "constraints \n")

print("CONST 4") # Hard precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each hard pred --> i start has to be > hard pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const4 += 1
            const4ik[i][a] = solver.Constraint(0, solver.infinity())
            const4ik[i][a].SetCoefficient(wi[i], l)
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    const4ik[i][a].SetCoefficient(xijt[i][j][t], t) # t*sum(xijt)
                for s in range(int(l)):
                    const4ik[i][a].SetCoefficient(xijt[k][j][s], -int(s+pij_duration.iloc[k,j])) # (s + pkj)*xkjs for each hard pred. CBC only accepts int as coeficient
print(num_const4, "constraints \n")

print("CONST 5") # Soft precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each soft pred --> i start has to be > soft pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const5 += 1
            const5ik[i][a] = solver.Constraint(0, solver.infinity())
            const5ik[i][a].SetCoefficient(wi[i], l)
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    const5ik[i][a].SetCoefficient(xijt[i][j][t], t) # t*sum(xijt)
                for s in range(int(l)):
                    const5ik[i][a].SetCoefficient(xijt[k][j][s], -int(s+pij_duration.iloc[k,j])) # (s + pkj)*xkjs for each soft pred. CBC only accepts int as coeficient
print(num_const5, "constraints \n")

print("CONST 6") # Exclusion of impossible machines --> if machine not allowed sum(xijt) = 0
for i in range(len(input_tasks)):
    num_const6 += 1
    const6i[i] = solver.Constraint(0, 0)
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            for t in range(int(l)):
                const6i[i].SetCoefficient(xijt[i][j][t], 1)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= sum(t*xijt) + sum(pij*sum(xij))
for i in range(len(input_tasks)):
    num_const7 += 1
    const7i[i] = solver.Constraint(0, solver.infinity()) # >= 0
    const7i[i].SetCoefficient(cmax, 1) # cmax
    for j in range(len(input_machines)):
        for t in range(int(l)):
            const7i[i].SetCoefficient(xijt[i][j][t], - t - int(pij_duration.iloc[i,j])) # -sum(t*xijt) - sum(pij*sum(xij))
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
parameters = pywraplp.MPSolverParameters()

if time_limit > 0:
    solver.SetTimeLimit(time_limit) # Set time limit

if maximum_gap_allowed > 0:
    parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP

solver.SetNumThreads(num_threads) # Use multi threading / cores
status = solver.Solve(parameters)

####################################################################################################
####################################################################################################
# 5 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)

# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        for t in range(int(l)):
            if xijt[i][j][t].solution_value() == 1: # Only add to DF results, skip blank values
                mip_allocation_output[row_num][1] = t # Start
                mip_allocation_output[row_num][2] = t + pij_duration.iloc[i,j] # Finish
                mip_allocation_output[row_num][3] = j # Machine
                mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
                mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in (solver.OPTIMAL, solver.FEASIBLE):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            for t in range(int(l)):
                if xijt[i][j][t].solution_value() == 1:
                    npv += tasks_profit.iloc[i,0]/((1+day_disc_rate)**t)
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(objective.BestBound() - solver.Objective().Value())/solver.Objective().Value() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CBC" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xijt" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = solver.NumVariables()
mip_stats_output[0][7] = solver.NumConstraints()
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/1000/3600 # Wall time in hours
mip_stats_output[0][10] = solver.Objective().Value() # Optimized value
mip_stats_output[0][11] = objective.BestBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 6 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 7 - Export linear model as txt
model = solver.ExportModelAsLpFormat(False)
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 8 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == solver.OPTIMAL:  # Optimal solution found 
    print("Optimized value: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
else:  # Optimal solution not found
    if status == solver.FEASIBLE:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in (solver.OPTIMAL, solver.FEASIBLE):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(objective.BestBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % solver.NumVariables())
    print("Total number of constraints: %i" % solver.NumConstraints())
    print("Total number of branch-and-bound nodes: %i \n" % solver.nodes())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % (solver.WallTime()/1000))
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    result_gap = abs(objective.BestBound() - solver.Objective().Value())/solver.Objective().Value() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE
    print("Gap: %.2f" % result_gap)
    
display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 9 - Update L value
l = int(result_makespan)

## 5 - CBC - xi

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value considered for optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - Create matrices to store variables
cmax = 0 # Makespan

xi = [] # Task start
for i in range(len(input_tasks)):
    xi.append(0)
    
yij = [] # 1 if machine j process task i. Otherwise, 0
for i in range(len(input_tasks)): # len(input_tasks) = number of tasks
    dim1 = []
    for j in range(len(input_machines)): # len(input_machines) = number of machines
        dim1.append(0)
    yij.append(dim1)

wik = [] # 1 if task i happens before task k. Otherwise, 0
for i in range(len(input_tasks)):
    dim1 = []
    for k in range(len(input_tasks)):
        dim1.append(0)
    wik.append(dim1)

####################################################################################################
####################################################################################################
# 3 - Create matrices to store constraints
const1i=[]
for i in range(len(input_tasks)):
    const1i.append(0)

const2i=[]
for i in range(len(input_tasks)):
    const2i.append(0)
    
const3i=[]
for i in range(len(input_tasks)):
    const3i.append(0)

const4i=[]
for i in range(len(input_tasks)):
    const4i.append(0)

const5ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(input_tasks)):
        dim1.append(0)
    const5ik.append(dim1)

const6ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(input_tasks)):
        dim1.append(0)
    const6ik.append(dim1)
    
const7ik=[]
for i in range(len(input_tasks)):
    dim1=[]
    for k in range(len(input_tasks)):
        dim1.append(0)
    const7ik.append(dim1)
    
const8ikj=[]
for i in range(len(input_tasks)):
    dim2 = []
    for k in range(len(input_tasks)):
        dim1 = []
        for j in range(len(input_machines)):
            dim1.append(0)
        dim2.append(dim1)
    const8ikj.append(dim2)

const9ikjh=[]
for i in range(len(input_tasks)):
    dim3 = []
    for k in range(len(input_tasks)):
        dim2 = []
        for j in range(len(input_machines)):
            dim1 = []
            for h in range(len(input_machines)):
                dim1.append(0)
            dim2.append(dim1)
        dim3.append(dim2)
    const9ikjh.append(dim3)

const10i=[]
for i in range(len(input_tasks)):
    const10i.append(0)

####################################################################################################
####################################################################################################
# 4 - MIP Model
# Objective function and variable
solver = pywraplp.Solver('UG scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective = solver.Objective()

# Objective function --> Not considering discount rate, only cash flow
cmax = solver.IntVar(0, l, 'cmax') # Makespan used to subtract period costs
for i in range(len(input_tasks)):
    xi[i] = solver.IntVar(0, l, 'xi'+str(i)) # L as an upper bound
    for j in range(len(input_machines)):
        yij[i][j] = solver.BoolVar('yij'+str(i)+'-'+str(j))
        objective.SetCoefficient(yij[i][j], int(tasks_profit.iloc[i,0])) # profit for cash flow calculation in obj function
    for k in range(len(input_tasks)):
        wik[i][k] = solver.BoolVar('wik'+str(i)+'-'+str(k))            
objective.SetCoefficient(cmax, -costs_period) # Objective function: - Cmax*costs_period (US$/d)
objective.SetMaximization()

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const8 = 0
num_const9 = 0
num_const10 = 0

print("CONST 1") # sum(yij) <= 1 if j in allowed_machines
for i in range(len(input_tasks)):
    num_const1 += 1
    const1i[i] = solver.Constraint(0, 1)
    for j in range(len(input_machines)):
        if j in allowed_machines.iloc[i,0]: # j in allowed machines
            const1i[i].SetCoefficient(yij[i][j], 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(yij) = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    const2i[i] = solver.Constraint(0, 0)
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]: # j NOT in allowed machines
            const2i[i].SetCoefficient(yij[i][j], 1)
print(num_const2, "constraints \n")

print("CONST 3") # xi <= l*sum(yij) --> relation between xi and xij
for i in range(len(input_tasks)):
    num_const3 += 1
    const3i[i] = solver.Constraint(-solver.infinity(), 0) # <=0
    const3i[i].SetCoefficient(xi[i], 1) # xi
    for j in range(len(input_machines)):
        const3i[i].SetCoefficient(yij[i][j], -l) # -sum(l*yij)
print(num_const3, "constraints \n")

print("CONST 4")  # Hard precedencies --> Qi_hard*sum(yij) <= sum(ykj) --> i only happens if ALL k hard predecessors happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has hard predecessors
        num_const4 += 1
        const4i[i] = solver.Constraint(-solver.infinity(), 0) # <= 0
        for j in range(len(input_machines)):
            const4i[i].SetCoefficient(yij[i][j], int(Qi_hard.iloc[i,1])) # Qi_hard*sum(yij) for all j
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                for j in range(len(input_machines)):
                    const4i[i].SetCoefficient(yij[k][j], -1) # - sum(ykj) for all j and k
print(num_const4, "constraints \n")

print("CONST 5")  # Hard precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const5 += 1
                const5ik[i][k] = solver.Constraint(-l, solver.infinity()) # >= -l
                const5ik[i][k].SetCoefficient(xi[i], 1) # xi
                const5ik[i][k].SetCoefficient(xi[k], -1) # -xk
                for j in range(len(input_machines)):
                    const5ik[i][k].SetCoefficient(yij[i][j], -l) # -l*sum(yij) for all j 
                    const5ik[i][k].SetCoefficient(yij[k][j], - int(pij_duration.iloc[k,j])) # - sum(pkj*ykj) for all j
print(num_const5, "constraints \n")

print("CONST 6")  # Soft precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_soft.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const6 += 1
                const6ik[i][k] = solver.Constraint(-l, solver.infinity()) # >= -l
                const6ik[i][k].SetCoefficient(xi[i], 1) # xi
                const6ik[i][k].SetCoefficient(xi[k], -1) # -xk
                for j in range(len(input_machines)):
                    const6ik[i][k].SetCoefficient(yij[i][j], -l) # -l*sum(yij) for all j 
                    const6ik[i][k].SetCoefficient(yij[k][j], - int(pij_duration.iloc[k,j])) # - sum(pkj*ykj) for all j
print(num_const6, "constraints \n")

print("CONST 7")  # No-overlapping 1 -->  For 2 tasks i and k: i happens before k OR k before i --> xi + sum(pij*yij) - xk - l*(1-wik) <= 0 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const7 += 1
            const7ik[i][k] = solver.Constraint(-solver.infinity(), l) # <= L
            const7ik[i][k].SetCoefficient(xi[i], 1) # xi            
            const7ik[i][k].SetCoefficient(xi[k], -1) # -xk
            for j in range(len(input_machines)):
                    const7ik[i][k].SetCoefficient(yij[i][j], int(pij_duration.iloc[i,j])) # + sum(pij*yij) for all j
            const7ik[i][k].SetCoefficient(wik[i][k], l) # L*wik
print(num_const7, "constraints \n")

print("CONST 8")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const8 += 1
                const8ikj[i][k][j] = solver.Constraint(-solver.infinity(), 1) # <= 1
                const8ikj[i][k][j].SetCoefficient(yij[i][j], 1) # yij
                const8ikj[i][k][j].SetCoefficient(yij[k][j], 1) # ykj
                const8ikj[i][k][j].SetCoefficient(wik[i][k], -1) # -wik
                const8ikj[i][k][j].SetCoefficient(wik[k][i], -1) # -wki
print(num_const8, "constraints \n")

print("CONST 9")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const9 += 1
                        const9ikjh[i][k][j][h] = solver.Constraint(-solver.infinity(), 2) # <= 2
                        const9ikjh[i][k][j][h].SetCoefficient(yij[i][j], 1) # yij
                        const9ikjh[i][k][j][h].SetCoefficient(yij[k][h], 1) # ykh
                        const9ikjh[i][k][j][h].SetCoefficient(wik[i][k], 1) # wik
                        const9ikjh[i][k][j][h].SetCoefficient(wik[k][i], 1) # wki
print(num_const9, "constraints \n")

print("CONST 10")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const10 += 1
    const10i[i] = solver.Constraint(0, solver.infinity()) # >= 0
    const10i[i].SetCoefficient(cmax, 1) # cmax
    const10i[i].SetCoefficient(xi[i], -1) # -xi
    for j in range(len(input_machines)):
        const10i[i].SetCoefficient(yij[i][j], - int(pij_duration.iloc[i,j])) # - sum(pij*yij) for all j
print(num_const10, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
parameters = pywraplp.MPSolverParameters()

if time_limit > 0:
    solver.SetTimeLimit(time_limit) # Set time limit

if maximum_gap_allowed > 0:
    parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP

solver.SetNumThreads(num_threads) # Use multi threading / cores
status = solver.Solve(parameters)

####################################################################################################
####################################################################################################
# 5 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if yij[i][j].solution_value() == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = xi[i].solution_value() # Start
            mip_allocation_output[row_num][2] = xi[i].solution_value() + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in (solver.OPTIMAL, solver.FEASIBLE):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if yij[i][j].solution_value() == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**xi[i].solution_value())
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(objective.BestBound() - solver.Objective().Value())/solver.Objective().Value() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CBC" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_profit" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = solver.NumVariables()
mip_stats_output[0][7] = solver.NumConstraints()
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/1000/3600 # Wall time in hours
mip_stats_output[0][10] = solver.Objective().Value() # Optimized value
mip_stats_output[0][11] = objective.BestBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 6 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 7 - Export linear model as txt
model = solver.ExportModelAsLpFormat(False)
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 8 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == solver.OPTIMAL:  # Optimal solution found 
    print("Optimized value: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
else:  # Optimal solution not found
    if status == solver.FEASIBLE:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.Objective().Value()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in (solver.OPTIMAL, solver.FEASIBLE):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(objective.BestBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % solver.NumVariables())
    print("Total number of constraints: %i" % solver.NumConstraints())
    print("Total number of branch-and-bound nodes: %i \n" % solver.nodes())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % (solver.WallTime()/1000))
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    print("Gap: %.2f" % result_gap)   

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 9 - Update L value
l = int(result_makespan)

## 6 - CP-SAT - xi optimizing makespan (l upper-bound alternative)

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
model = cp_model.CpModel()

# Variables
cmax = model.NewIntVar(0, int(l), 'cmax') # Consider L already as an upper bound to makespan
xi = [model.NewIntVar(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[model.NewBoolVar('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[model.NewBoolVar('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 
# print(np.matrix(yij)) #NP command for print in matrix format

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Minimize makespan
model.Minimize(cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # sum(yij) = 1 --> ALL tasks will be allocated
for i in range(len(input_tasks)):
    num_const1 += 1
    model.Add(sum(yij[i]) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # yij = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]: # j NOT in allowed machines
            model.Add(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3")  # Precedencies (hard + soft) --> xi >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const3 += 1
                model.Add(xi[i] >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const3, "constraints \n")

print("CONST 4")  # No-overlapping 1 -->  xij + pij <= xkj + l*(1-wik) (k is another task, not a predecessor) --> For 2 tasks i and k: i happens before k OR k before i
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const4 += 1
            model.Add(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const4, "constraints \n")

print("CONST 5")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const5 += 1
                model.Add(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const5, "constraints \n")

print("CONST 6")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const6 += 1
                        model.Add(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const7 += 1
    model.Add(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
solver = cp_model.CpSolver() # Define solver

if time_limit > 0:
    solver.parameters.max_time_in_seconds = time_limit/1000 # Set time limit

# if maximum_gap_allowed > 0: # Not available on CP-SAT
#     solver.parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP # Not available on CP-SAT

solver.parameters.num_search_workers = num_threads # Use multi threading / cores
status = solver.Solve(model) # Solve
status = solver.StatusName(status) # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if solver.Value(yij[i][j]) == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = solver.Value(xi[i]) # Start
            mip_allocation_output[row_num][2] = solver.Value(xi[i]) + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in ("OPTIMAL", "FEASIBLE"):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if solver.Value(yij[i][j]) == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**solver.Value(xi[i]))
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.BestObjectiveBound() - solver.ObjectiveValue())/solver.ObjectiveValue() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CP-SAT" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_makespan" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = num_var_total
mip_stats_output[0][7] = num_const_total
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/3600 # Wall time in hours
mip_stats_output[0][10] = solver.ObjectiveValue() # Optimized value
mip_stats_output[0][11] = solver.BestObjectiveBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
model = str(model.ModelStats()) + "\n\n" + str(solver.ResponseProto()) # CP-SAT Model stats
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == "OPTIMAL":  # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
else:  # Optimal solution not found
    if status == "FEASIBLE":
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in ("OPTIMAL", "FEASIBLE"):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.BestObjectiveBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % num_var_total)
    print("Total number of constraints: %i" % num_const_total)
    print("Total number of branch-and-bound branches: %i \n" % solver.NumBranches())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.WallTime())
#     if maximum_gap_allowed > 0: # Not available on CP-SAT
#         print("Gap limit: %.2f" % maximum_gap_allowed) # Not available on CP-SAT
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

## 7 - CP-SAT - xijt (time-indexed)

In [ ]:
####################################################################################################
####################################################################################################
# 13 - Tasks profit (i) vs j vs t (same from ) --> from PREPARE INPU --> Rerun required if running code below multiple times (updating L-value)
tasks_profit_xit = pd.DataFrame(np.tile(tasks_profit.to_numpy(),int(l))) # DF with profit repeated l columns
discount_factors_xit = np.fromfunction(lambda i, j: 1/(1+day_disc_rate)**j, (len(input_tasks), int(l))) # Discount rates multiplying factors for each
discount_factors_xit = pd.DataFrame(discount_factors_xit) # DF with previous line data
tasks_profit_xijt = (tasks_profit_xit*discount_factors_xit).astype(int) # Multiply profit and discount factors
tasks_profit_xijt['0'] = tasks_profit_xijt.values.tolist() # Group columns into lists into one cell
tasks_profit_xijt.drop(tasks_profit_xijt.columns.difference(['0']), 1, inplace=True) # Drop all columns but the list one
tasks_profit_xijt = pd.DataFrame(np.tile(tasks_profit_xijt.to_numpy(),len(input_machines))) # Repeat the list column (with discounted profit) j-machine times
tasks_profit_xijt = tasks_profit_xijt.values.tolist() # Get list from df

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value considered for optimization: %i \n" % l)
l_input = l

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
model = cp_model.CpModel()

# Variables
cmax = model.NewIntVar(0, int(l), 'cmax') # Makespan used to subtract period costs
xijt = [[[model.NewBoolVar('xij_%i_%i_%i' % (i, j, t)) for t in range(int(l))] for j in range(len(input_machines))] for i in range(len(input_tasks))]
wi = [model.NewBoolVar('wi_%i' % (i)) for i in range(len(input_tasks))]

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xijt[0][0])*len(xijt[0])*len(xijt) # xijt
num_var3 = len(wi) # wi
num_var_total = num_var1 + num_var2 + num_var3 # Total number of variables

# Objective function --> Maximize NPV
model.Maximize(np.multiply(xijt,tasks_profit_xijt).sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # wi + sum(xijt) = 1 --> wi = 1 => tasks not processed. Otherwise: wi = 0
for i in range(len(input_tasks)):
    num_const1 += 1
    model.Add(wi[i] + np.sum(np.array(xijt[i])) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(xijs) <= 1 --> For each machine, in each time t only one task can occupy the machine
for t in range(int(l)):
    for j in range(len(input_machines)):
        num_const2 += 1
        sum_xijs = []
        for i in range(len(input_tasks)):
                for s in range(max(int(t-pij_duration.iloc[i,j]+1),0),t+1): # s is the time window that includes processing time
                    sum_xijs.append(xijt[i][j][s]) # append adds 1 element, extend adds a list of elements
        model.Add(sum(sum_xijs) <= 1)
print(num_const2, "constraints \n")

print("CONST 3") # Hard precedencies --> Qi_hard*sum(xijt) <= sum(xkjt) --> i only happens if ALL k hard predecessors happens  All hard pred as requirement for xijt = 1. Otherwise, xijt = 0
for i in range(len(input_tasks)):
    xijt_forall_k = []
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            xijt_forall_k.extend(xijt[k])
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        num_const3 += 1
        model.Add(Qi_hard.iloc[i,1]*np.sum(xijt[i]) <= sum(np.array(xijt_forall_k).flatten())) # Flatten used to change 2D list to 1D list
print(num_const3, "constraints \n")

print("CONST 4") # Hard precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each hard pred --> i start has to be > hard pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const4 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            model.Add(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const4, "constraints \n")

print("CONST 5") # Soft precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each soft pred --> i start has to be > soft pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const5 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            model.Add(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const5, "constraints \n")

print("CONST 6") # Exclusion of impossible machines --> if machine not allowed sum(xijt) = 0
for i in range(len(input_tasks)):
    num_const6 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            model.Add(sum(np.array(xijt[i][j]).flatten()) == 0)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= sum(t*xijt) + sum(pij*sum(xij))
for i in range(len(input_tasks)):
    num_const7 += 1
    t_multi_xijt_forall_tj = []
    xijt_multi_pij_duration = []
    for j in range(len(input_machines)):
        for t in range(int(l)):
            t_multi_xijt_forall_tj.append(t*xijt[i][j][t]) # Start time
            xijt_multi_pij_duration.append(xijt[i][j][t]*pij_duration.iloc[i,j]) # Duration
    model.Add(cmax >= sum(np.array(t_multi_xijt_forall_tj).flatten()) + sum(np.array(xijt_multi_pij_duration).flatten()))
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
solver = cp_model.CpSolver() # Define solver

if time_limit > 0:
    solver.parameters.max_time_in_seconds = time_limit/1000 # Set time limit

# if maximum_gap_allowed > 0: # Not available on CP-SAT
#     parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP # Not available on CP-SAT

solver.parameters.num_search_workers = num_threads # Use multi threading / cores
status = solver.Solve(model) # Solve
status = solver.StatusName(status) # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)

# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        for t in range(int(l)):
            if solver.Value(xijt[i][j][t]) == 1: # Only add to DF results, skip blank values
                mip_allocation_output[row_num][1] = t # Start
                mip_allocation_output[row_num][2] = t + pij_duration.iloc[i,j] # Finish
                mip_allocation_output[row_num][3] = j # Machine
                mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
                mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in ("OPTIMAL", "FEASIBLE"):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            for t in range(int(l)):
                if solver.Value(xijt[i][j][t]) == 1:
                    npv += tasks_profit.iloc[i,0]/((1+day_disc_rate)**t)
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.BestObjectiveBound() - solver.ObjectiveValue())/solver.ObjectiveValue() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE
    
# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CP-SAT" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xijt" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = num_var_total
mip_stats_output[0][7] = num_const_total
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/3600 # Wall time in hours
mip_stats_output[0][10] = solver.ObjectiveValue() # Optimized value
mip_stats_output[0][11] = solver.BestObjectiveBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
model = str(model.ModelStats()) + "\n\n" + str(solver.ResponseProto()) # CP-SAT Model stats
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == "OPTIMAL":  # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
else:  # Optimal solution not found
    if status == "FEASIBLE":
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in ("OPTIMAL", "FEASIBLE"):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.BestObjectiveBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % num_var_total)
    print("Total number of constraints: %i" % num_const_total)
    print("Total number of branch-and-bound branches: %i \n" % solver.NumBranches())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.WallTime())
#     if maximum_gap_allowed > 0: # Not available on CP-SAT
#         print("Gap limit: %.2f" % maximum_gap_allowed) # Not available on CP-SAT
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

## 8 - CP-SAT - xi

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
model = cp_model.CpModel()

# Variables
cmax = model.NewIntVar(0, int(l), 'cmax') # Makespan used to subtract period costs
xi = [model.NewIntVar(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[model.NewBoolVar('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[model.NewBoolVar('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Not considering discount rate yet, only cash flow
model.Maximize(np.multiply(yij,tasks_profit_matrix).to_numpy().sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const8 = 0
num_const9 = 0
num_const10 = 0
num_const_total = 0

print("CONST 1") # sum(yij) <= 1 if j in allowed_machines
for i in range(len(input_tasks)):
    num_const1 += 1
    model.Add(sum(yij[i]) <= 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(yij) = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            model.Add(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3") # xi <= l*sum(yij) --> relation between xi and xij
for i in range(len(input_tasks)):
    num_const3 += 1
    model.Add(xi[i] <= l*sum(yij[i])) # l value limits xij
print(num_const3, "constraints \n")

print("CONST 4")  # Hard precedencies --> Qi_hard*sum(yij) <= sum(ykj) --> i only happens if ALL k hard predecessors happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has hard predecessors
        num_const4 += 1
        yij_forall_k = []
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:          
                yij_forall_k.extend(yij[k])
        model.Add(Qi_hard.iloc[i,1]*sum(yij[i]) <= sum(yij_forall_k)) # yij = 1 only if all pred processed
print(num_const4, "constraints \n")

print("CONST 5")  # Hard precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const5 += 1
                model.Add(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const5, "constraints \n")

print("CONST 6")  # Soft precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_soft.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const6 += 1
                model.Add(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const6, "constraints \n")

print("CONST 7")  # No-overlapping 1 -->  For 2 tasks i and k: i happens before k OR k before i --> xi + sum(pij*yij) <= xk + l*(1-wik) (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const7 += 1
            model.Add(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const7, "constraints \n")

print("CONST 8")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const8 += 1
                model.Add(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const8, "constraints \n")

print("CONST 9")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const9 += 1
                        model.Add(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const9, "constraints \n")

print("CONST 10")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const10 += 1
    model.Add(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const10, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")
solver = cp_model.CpSolver() # Define solver

if time_limit > 0:
    solver.parameters.max_time_in_seconds = time_limit/1000 # Set time limit

# if maximum_gap_allowed > 0: # Not available on CP-SAT
#     solver.parameters.RELATIVE_MIP_GAP = maximum_gap_allowed # Set maximum GAP # Not available on CP-SAT

solver.parameters.num_search_workers = num_threads # Use multi threading / cores
status = solver.Solve(model) # Solve
status = solver.StatusName(status) # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 + num_const8 + num_const9 + num_const10 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if solver.Value(yij[i][j]) == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = solver.Value(xi[i]) # Start
            mip_allocation_output[row_num][2] = solver.Value(xi[i]) + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if status in ("OPTIMAL", "FEASIBLE"):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if solver.Value(yij[i][j]) == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**solver.Value(xi[i]))
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.BestObjectiveBound() - solver.ObjectiveValue())/solver.ObjectiveValue() # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CP-SAT" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_profit" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = num_var_total
mip_stats_output[0][7] = num_const_total
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.WallTime()/3600 # Wall time in hours
mip_stats_output[0][10] = solver.ObjectiveValue() # Optimized value
mip_stats_output[0][11] = solver.BestObjectiveBound() # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
model = str(model.ModelStats()) + "\n\n" + str(solver.ResponseProto()) # CP-SAT Model stats
f = open(outfile + str('.txt'), 'w')
f.writelines(model)
f.close()
print("\nMIP model saved as: " + outfile + str('.txt'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if status == "OPTIMAL":  # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
else:  # Optimal solution not found
    if status == "FEASIBLE":
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.ObjectiveValue()).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if status in ("OPTIMAL", "FEASIBLE"):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.BestObjectiveBound()).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % num_var_total)
    print("Total number of constraints: %i" % num_const_total)
    print("Total number of branch-and-bound branches: %i \n" % solver.NumBranches())
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.WallTime())
#     if maximum_gap_allowed > 0: # Not available on CP-SAT
#         print("Gap limit: %.2f" % maximum_gap_allowed) # Not available on CP-SAT
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

## 9 - CPLEX - xi optimizing makespan (l upper-bound alternative)

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax')
xi = [mdl.integer_var(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[mdl.binary_var('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[mdl.binary_var('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Minimize makespan
mdl.minimize(cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # sum(yij) = 1 --> ALL tasks will be allocated
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(sum(yij[i]) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # yij = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]: # j NOT in allowed machines
            mdl.add_constraint(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3")  # Precedencies (hard + soft) --> xi >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const3 += 1
                mdl.add_constraint(xi[i] >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const3, "constraints \n")

print("CONST 4")  # No-overlapping 1 -->  xij + pij <= xkj + l*(1-wik) (k is another task, not a predecessor) --> For 2 tasks i and k: i happens before k OR k before i
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const4 += 1
            mdl.add_constraint(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const4, "constraints \n")

print("CONST 5")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const5 += 1
                mdl.add_constraint(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const5, "constraints \n")

print("CONST 6")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const6 += 1
                        mdl.add_constraint(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const7 += 1
    mdl.add_constraint(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")

if time_limit > 0:
    mdl.set_time_limit(time_limit/1000) # Set time limit in seconds

if maximum_gap_allowed > 0:
    mdl.parameters.mip.tolerances.mipgap(maximum_gap_allowed) # Set maximum GAP

mdl.parameters.threads(num_threads) # Use multi threading / cores

solver = mdl.solve() # Solve
status = solver.solve_details.status # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if yij[i][j].solution_value == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = xi[i].solution_value # Start
            mip_allocation_output[row_num][2] = xi[i].solution_value + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if ("optimal" in status) or ("feasible" in status):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if yij[i][j].solution_value == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**xi[i].solution_value)
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.solve_details.best_bound - solver.objective_value)/solver.solve_details.best_bound # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CPLEX" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_makespan" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = mdl.number_of_variables
mip_stats_output[0][7] = mdl.number_of_constraints
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.solve_details.time/3600 # Wall time in hours
mip_stats_output[0][10] = solver.objective_value # Optimized value
mip_stats_output[0][11] = solver.solve_details.best_bound # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if "optimal" in status: # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.objective_value).replace(',', ' '))
else:  # Optimal solution not found
    if "feasible" in status:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.objective_value).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if ("optimal" in status) or ("feasible" in status):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.solve_details.best_bound).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % mdl.number_of_variables)
    print("Total number of constraints: %i" % mdl.number_of_constraints)
    print("Total number of branch-and-bound branches: %i \n" % solver.solve_details.nb_nodes_processed)
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.solve_details.time)
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

# Other CPLEX stats
# mdl.print_information()
# mdl.print_solution()
# mdl.report()
# gap = solver.solve_details.mip_relative_gap

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

In [ ]:
# JUST TO EXPORT MODEL AS LP


####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax')
xi = [mdl.integer_var(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[mdl.binary_var('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[mdl.binary_var('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Minimize makespan
mdl.minimize(cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # sum(yij) = 1 --> ALL tasks will be allocated
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(sum(yij[i]) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # yij = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]: # j NOT in allowed machines
            mdl.add_constraint(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3")  # Precedencies (hard + soft) --> xi >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const3 += 1
                mdl.add_constraint(xi[i] >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const3, "constraints \n")

print("CONST 4")  # No-overlapping 1 -->  xij + pij <= xkj + l*(1-wik) (k is another task, not a predecessor) --> For 2 tasks i and k: i happens before k OR k before i
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const4 += 1
            mdl.add_constraint(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const4, "constraints \n")

print("CONST 5")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const5 += 1
                mdl.add_constraint(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const5, "constraints \n")

print("CONST 6")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const6 += 1
                        mdl.add_constraint(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const7 += 1
    mdl.add_constraint(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const7, "constraints \n")

# Output filename
outfile = str("CPLEX") + str('-') + str("xi_makespan") + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))

print("\nTotal number of variables: %i" % mdl.number_of_variables)
print("Total number of constraints: %i" % mdl.number_of_constraints)

## 10 - CPLEX - xijt (time-indexed)

In [ ]:
####################################################################################################
####################################################################################################
# 13 - Tasks profit (i) vs j vs t (same from ) --> from PREPARE INPU --> Rerun required if running code below multiple times (updating L-value)
tasks_profit_xit = pd.DataFrame(np.tile(tasks_profit.to_numpy(),int(l))) # DF with profit repeated l columns
discount_factors_xit = np.fromfunction(lambda i, j: 1/(1+day_disc_rate)**j, (len(input_tasks), int(l))) # Discount rates multiplying factors for each
discount_factors_xit = pd.DataFrame(discount_factors_xit) # DF with previous line data
tasks_profit_xijt = (tasks_profit_xit*discount_factors_xit).astype(int) # Multiply profit and discount factors
tasks_profit_xijt['0'] = tasks_profit_xijt.values.tolist() # Group columns into lists into one cell
tasks_profit_xijt.drop(tasks_profit_xijt.columns.difference(['0']), 1, inplace=True) # Drop all columns but the list one
tasks_profit_xijt = pd.DataFrame(np.tile(tasks_profit_xijt.to_numpy(),len(input_machines))) # Repeat the list column (with discounted profit) j-machine times
tasks_profit_xijt = tasks_profit_xijt.values.tolist() # Get list from df

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600*24*1000 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1
    
####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax') # Makespan used to subtract period costs
xijt = [[[mdl.binary_var('xij_%i_%i_%i' % (i, j, t)) for t in range(int(l))] for j in range(len(input_machines))] for i in range(len(input_tasks))]
wi = [mdl.binary_var('wi_%i' % (i)) for i in range(len(input_tasks))]

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xijt[0][0])*len(xijt[0])*len(xijt) # xijt
num_var3 = len(wi) # wi
num_var_total = num_var1 + num_var2 + num_var3 # Total number of variables

# Objective function --> Maximize NPV
mdl.maximize(np.multiply(xijt,tasks_profit_xijt).sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # wi + sum(xijt) = 1 --> wi = 1 => tasks not processed. Otherwise: wi = 0
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(wi[i] + np.sum(np.array(xijt[i])) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(xijs) <= 1 --> For each machine, in each time t only one task can occupy the machine
for t in range(int(l)):
    for j in range(len(input_machines)):
        num_const2 += 1
        sum_xijs = []
        for i in range(len(input_tasks)):
                for s in range(max(int(t-pij_duration.iloc[i,j]+1),0),t+1): # s is the time window that includes processing time
                    sum_xijs.append(xijt[i][j][s]) # append adds 1 element, extend adds a list of elements
        mdl.add_constraint(sum(sum_xijs) <= 1)
print(num_const2, "constraints \n")

print("CONST 3") # Hard precedencies --> Qi_hard*sum(xijt) <= sum(xkjt) --> i only happens if ALL k hard predecessors happens  All hard pred as requirement for xijt = 1. Otherwise, xijt = 0
for i in range(len(input_tasks)):
    xijt_forall_k = []
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            xijt_forall_k.extend(xijt[k])
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        num_const3 += 1
        mdl.add_constraint(Qi_hard.iloc[i,1]*np.sum(xijt[i]) <= sum(np.array(xijt_forall_k).flatten())) # Flatten used to change 2D list to 1D list
print(num_const3, "constraints \n")

print("CONST 4") # Hard precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each hard pred --> i start has to be > hard pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const4 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            mdl.add_constraint(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const4, "constraints \n")

print("CONST 5") # Soft precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each soft pred --> i start has to be > soft pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const5 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            mdl.add_constraint(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const5, "constraints \n")

print("CONST 6") # Exclusion of impossible machines --> if machine not allowed sum(xijt) = 0
for i in range(len(input_tasks)):
    num_const6 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            mdl.add_constraint(sum(np.array(xijt[i][j]).flatten()) == 0)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= sum(t*xijt) + sum(pij*sum(xij))
for i in range(len(input_tasks)):
    num_const7 += 1
    t_multi_xijt_forall_tj = []
    xijt_multi_pij_duration = []
    for j in range(len(input_machines)):
        for t in range(int(l)):
            t_multi_xijt_forall_tj.append(t*xijt[i][j][t]) # Start time
            xijt_multi_pij_duration.append(xijt[i][j][t]*pij_duration.iloc[i,j]) # Duration
    mdl.add_constraint(cmax >= sum(np.array(t_multi_xijt_forall_tj).flatten()) + sum(np.array(xijt_multi_pij_duration).flatten()))
print(num_const7, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")

if time_limit > 0:
    mdl.set_time_limit(time_limit/1000) # Set time limit in seconds

if maximum_gap_allowed > 0:
    mdl.parameters.mip.tolerances.mipgap(maximum_gap_allowed) # Set maximum GAP

mdl.parameters.threads(num_threads) # Use multi threading / cores

solver = mdl.solve() # Solve
status = solver.solve_details.status # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        for t in range(int(l)):
            if xijt[i][j][t].solution_value == 1: # Only add to DF results, skip blank values
                mip_allocation_output[row_num][1] = t # Start
                mip_allocation_output[row_num][2] = t + pij_duration.iloc[i,j] # Finish
                mip_allocation_output[row_num][3] = j # Machine
                mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
                mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if ("optimal" in status) or ("feasible" in status):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            for t in range(int(l)):
                if xijt[i][j][t].solution_value == 1:
                    npv += tasks_profit.iloc[i,0]/((1+day_disc_rate)**t)
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.solve_details.best_bound - solver.objective_value)/solver.solve_details.best_bound # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CPLEX" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xijt" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = mdl.number_of_variables
mip_stats_output[0][7] = mdl.number_of_constraints
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.solve_details.time/3600 # Wall time in hours
mip_stats_output[0][10] = solver.objective_value # Optimized value
mip_stats_output[0][11] = solver.solve_details.best_bound # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if "optimal" in status: # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.objective_value).replace(',', ' '))
else:  # Optimal solution not found
    if "feasible" in status:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.objective_value).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if ("optimal" in status) or ("feasible" in status):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.solve_details.best_bound).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % mdl.number_of_variables)
    print("Total number of constraints: %i" % mdl.number_of_constraints)
    print("Total number of branch-and-bound branches: %i \n" % solver.solve_details.nb_nodes_processed)
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.solve_details.time)
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

# Other CPLEX stats
# mdl.print_information()
# mdl.print_solution()
# mdl.report()
# gap = solver.solve_details.mip_relative_gap

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

In [ ]:
# JUST TO EXPORT MODEL AS LP


####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results
   
####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax') # Makespan used to subtract period costs
xijt = [[[mdl.binary_var('xij_%i_%i_%i' % (i, j, t)) for t in range(int(l))] for j in range(len(input_machines))] for i in range(len(input_tasks))]
wi = [mdl.binary_var('wi_%i' % (i)) for i in range(len(input_tasks))]

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xijt[0][0])*len(xijt[0])*len(xijt) # xijt
num_var3 = len(wi) # wi
num_var_total = num_var1 + num_var2 + num_var3 # Total number of variables

# Objective function --> Maximize NPV
mdl.maximize(np.multiply(xijt,tasks_profit_xijt).sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const_total = 0

print("CONST 1") # wi + sum(xijt) = 1 --> wi = 1 => tasks not processed. Otherwise: wi = 0
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(wi[i] + np.sum(np.array(xijt[i])) == 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(xijs) <= 1 --> For each machine, in each time t only one task can occupy the machine
for t in range(int(l)):
    for j in range(len(input_machines)):
        num_const2 += 1
        sum_xijs = []
        for i in range(len(input_tasks)):
                for s in range(max(int(t-pij_duration.iloc[i,j]+1),0),t+1): # s is the time window that includes processing time
                    sum_xijs.append(xijt[i][j][s]) # append adds 1 element, extend adds a list of elements
        mdl.add_constraint(sum(sum_xijs) <= 1)
print(num_const2, "constraints \n")

print("CONST 3") # Hard precedencies --> Qi_hard*sum(xijt) <= sum(xkjt) --> i only happens if ALL k hard predecessors happens  All hard pred as requirement for xijt = 1. Otherwise, xijt = 0
for i in range(len(input_tasks)):
    xijt_forall_k = []
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            xijt_forall_k.extend(xijt[k])
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        num_const3 += 1
        mdl.add_constraint(Qi_hard.iloc[i,1]*np.sum(xijt[i]) <= sum(np.array(xijt_forall_k).flatten())) # Flatten used to change 2D list to 1D list
print(num_const3, "constraints \n")

print("CONST 4") # Hard precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each hard pred --> i start has to be > hard pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const4 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            mdl.add_constraint(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const4, "constraints \n")

print("CONST 5") # Soft precedencies --> sum(t*xijt) + l*wi >= (s + pkj)*xkjs for each soft pred --> i start has to be > soft pred start or i does not happen
for i in range(len(input_tasks)):
    for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
        k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
        if k!=-1:
            num_const5 += 1
            t_multi_xijt_forall_tj = []
            s_multi_xkjs_forall_sj = []
            for j in range(len(input_machines)):
                for t in range(int(l)):
                    t_multi_xijt_forall_tj.append(t*xijt[i][j][t])
                for s in range(int(l)):
                    s_multi_xkjs_forall_sj.append((s+pij_duration.iloc[k,j])*xijt[k][j][s])
            mdl.add_constraint(sum(np.array(t_multi_xijt_forall_tj).flatten()) + l*wi[i] >= sum(np.array(s_multi_xkjs_forall_sj).flatten()))
print(num_const5, "constraints \n")

print("CONST 6") # Exclusion of impossible machines --> if machine not allowed sum(xijt) = 0
for i in range(len(input_tasks)):
    num_const6 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            mdl.add_constraint(sum(np.array(xijt[i][j]).flatten()) == 0)
print(num_const6, "constraints \n")

print("CONST 7")  # Define makespan -->  Cmax >= sum(t*xijt) + sum(pij*sum(xij))
for i in range(len(input_tasks)):
    num_const7 += 1
    t_multi_xijt_forall_tj = []
    xijt_multi_pij_duration = []
    for j in range(len(input_machines)):
        for t in range(int(l)):
            t_multi_xijt_forall_tj.append(t*xijt[i][j][t]) # Start time
            xijt_multi_pij_duration.append(xijt[i][j][t]*pij_duration.iloc[i,j]) # Duration
    mdl.add_constraint(cmax >= sum(np.array(t_multi_xijt_forall_tj).flatten()) + sum(np.array(xijt_multi_pij_duration).flatten()))
print(num_const7, "constraints \n")
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

# Output filename
outfile = str("CPLEX") + str('-') + str("xijt") + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))

print("\nTotal number of variables: %i" % mdl.number_of_variables)
print("Total number of constraints: %i" % mdl.number_of_constraints)

## 11 - CPLEX - xi

In [ ]:
####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax') # Makespan used to subtract period costs
xi = [mdl.integer_var(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[mdl.binary_var('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[mdl.binary_var('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Not considering discount rate yet, only cash flow
mdl.maximize(np.multiply(yij,tasks_profit_matrix).to_numpy().sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const8 = 0
num_const9 = 0
num_const10 = 0
num_const_total = 0

print("CONST 1") # sum(yij) <= 1 if j in allowed_machines
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(sum(yij[i]) <= 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(yij) = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            mdl.add_constraint(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3") # xi <= l*sum(yij) --> relation between xi and xij
for i in range(len(input_tasks)):
    num_const3 += 1
    mdl.add_constraint(xi[i] <= l*sum(yij[i])) # l value limits xij
print(num_const3, "constraints \n")

print("CONST 4")  # Hard precedencies --> Qi_hard*sum(yij) <= sum(ykj) --> i only happens if ALL k hard predecessors happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has hard predecessors
        num_const4 += 1
        yij_forall_k = []
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:          
                yij_forall_k.extend(yij[k])
        mdl.add_constraint(Qi_hard.iloc[i,1]*sum(yij[i]) <= sum(yij_forall_k)) # yij = 1 only if all pred processed
print(num_const4, "constraints \n")

print("CONST 5")  # Hard precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const5 += 1
                mdl.add_constraint(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const5, "constraints \n")

print("CONST 6")  # Soft precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_soft.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const6 += 1
                mdl.add_constraint(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const6, "constraints \n")

print("CONST 7")  # No-overlapping 1 -->  For 2 tasks i and k: i happens before k OR k before i --> xi + sum(pij*yij) <= xk + l*(1-wik) (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const7 += 1
            mdl.add_constraint(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const7, "constraints \n")

print("CONST 8")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const8 += 1
                mdl.add_constraint(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const8, "constraints \n")

print("CONST 9")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const9 += 1
                        mdl.add_constraint(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const9, "constraints \n")

print("CONST 10")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const10 += 1
    mdl.add_constraint(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const10, "constraints \n")

# Solve
print(">>>>>>>>>> SOLVING <<<<<<<<<<")

if time_limit > 0:
    mdl.set_time_limit(time_limit/1000) # Set time limit in seconds

if maximum_gap_allowed > 0:
    mdl.parameters.mip.tolerances.mipgap(maximum_gap_allowed) # Set maximum GAP

mdl.parameters.threads(num_threads) # Use multi threading / cores

solver = mdl.solve() # Solve
status = solver.solve_details.status # Get status
num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints

####################################################################################################
####################################################################################################
# 3 - Generate DF with results (sheet1 = stats, sheet2 = allocation) and print results
# Create output matrices
mip_stats_output = [] # Stats
for i in range(1):
    dim1 = []
    for col_num in range(15): # 15 stats to be reported
         dim1.append('')
    mip_stats_output.append(dim1)

mip_allocation_output=[] # Allocation
for i in range(len(input_tasks)):
    dim1 = []
    for col_num in range(6): # 'IDNUM', 'MACHINE', 'START xij', 'DURATION pij', 'Finish' , 'PROFIT'
        dim1.append('')
    mip_allocation_output.append(dim1)
    
# Populate 2D matrices with solution values
# Populate allocation
row_num = 0 # Row index
for i in range(len(input_tasks)):
    mip_allocation_output[row_num][0] = i # IDNUM of task
    for j in range(len(input_machines)):
        if yij[i][j].solution_value == 1: # Only add to DF results, skip blank values
            mip_allocation_output[row_num][1] = xi[i].solution_value # Start
            mip_allocation_output[row_num][2] = xi[i].solution_value + pij_duration.iloc[i,j] # Finish
            mip_allocation_output[row_num][3] = j # Machine
            mip_allocation_output[row_num][4] = pij_duration.iloc[i,j] # Duration (pij)
            mip_allocation_output[row_num][5] = tasks_profit_matrix.iloc[i,j] # Profit
    row_num += 1 # Add an extra row at end of 2D output matrix

# Create output DF for Allocation
mip_allocation_output = np.array(mip_allocation_output) # Move to np array to export directly to pandas DF
mip_allocation_output_df = pd.DataFrame(mip_allocation_output)
mip_allocation_output_df.columns = ['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit'] # Rename columns

# Calculate NPV and GAP (needs mip_allocation_output ready)
npv = 0
if ("optimal" in status) or ("feasible" in status):
    for i in range(len(input_tasks)):
        for j in range(len(input_machines)):
            if yij[i][j].solution_value == 1:
                npv += tasks_profit_matrix.iloc[i,j]/((1+day_disc_rate)**xi[i].solution_value)
    result_makespan = pd.to_numeric(mip_allocation_output_df['Finish']).max() # Calculate makespan based on mip_output_df
    npv += - result_makespan*costs_period # Period costs subtracting NPV
    
    result_gap = abs(solver.solve_details.best_bound - solver.objective_value)/solver.solve_details.best_bound # GAP = (BEST BOUND - OBJECTIVE)/ OBJECTIVE

# Populate stats
mip_stats_output[0][0] = inputfile[0] # Input xlsx file used
mip_stats_output[0][1] = len(input_tasks) # Number of tasks
mip_stats_output[0][2] = len(input_machines) # Number of machines
mip_stats_output[0][3] = l_input # l value used as input
mip_stats_output[0][4] = "CPLEX" # Solver: CBC; CP-SAT; CPLEX
mip_stats_output[0][5] = "xi_profit" # Model: xi_makespan; xijt; xi_profit
mip_stats_output[0][6] = mdl.number_of_variables
mip_stats_output[0][7] = mdl.number_of_constraints
mip_stats_output[0][8] = time_limit/1000/3600 # Time limit in hours
mip_stats_output[0][9] = solver.solve_details.time/3600 # Wall time in hours
mip_stats_output[0][10] = solver.objective_value # Optimized value
mip_stats_output[0][11] = solver.solve_details.best_bound # Best bound
mip_stats_output[0][12] = result_gap # GAP
mip_stats_output[0][13] = npv # NPV
mip_stats_output[0][14] = result_makespan # Makespan

# Create output DF for Stats
mip_stats_output = np.array(mip_stats_output) # Move to np array to export directly to pandas DF
mip_stats_output_df = pd.DataFrame(mip_stats_output)
mip_stats_output_df.columns = ['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)'] # Rename columns

# Convert allocation output DF to int (so as to have number in Excel)
mip_allocation_output_df_xlsx = mip_allocation_output_df.copy()
mip_allocation_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.applymap(str) # Convert object to str
mip_allocation_output_df_xlsx = mip_allocation_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_stats_output_df)# Sheet 1 - Stats
sh2 = pd.DataFrame(mip_allocation_output_df_xlsx) # Sheet 2 - Allocation

# Output filename
outfile = str(mip_stats_output[0][4]) + str('-') + str(mip_stats_output[0][5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

# Export output in Excel file
writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'STATS' # Sheet names
sh2_name = 'ALLOCATION' # Sheet names

sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()
print("\nResults saved as: " + outfile + str('.xlsx'))

####################################################################################################
####################################################################################################
# 4 - Plotly export (image)
mip_allocation_output_df_xlsx['Machine'] = mip_allocation_output_df_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
fig = ff.create_gantt(mip_allocation_output_df_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

fig.write_image(outfile + str('.png'))# Save as image
print("\nImage saved as: " + outfile + str('.png'))

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))

####################################################################################################
####################################################################################################
# 6 - Print results
# Solver results
print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
if "optimal" in status: # Optimal solution found
    print("Optimized value: {:,.0f}".format(solver.objective_value).replace(',', ' '))
else:  # Optimal solution not found
    if "feasible" in status:
        print("Feasible (sub-optimal solution) found: {:,.0f}".format(solver.objective_value).replace(',', ' '))
    else:
        print("No feasible solution could be found.")
if ("optimal" in status) or ("feasible" in status):
# Report solution NPV
    print("\nNPV:  {:,.0f} \n".format(npv).replace(',', ' '))
# Report other stats
    print("Best bound: {:,.0f}".format(solver.solve_details.best_bound).replace(',', ' '))
    print("\nMakespan: %i" % result_makespan)
    print("\nTotal number of variables: %i" % mdl.number_of_variables)
    print("Total number of constraints: %i" % mdl.number_of_constraints)
    print("Total number of branch-and-bound branches: %i \n" % solver.solve_details.nb_nodes_processed)
    if time_limit > 0:
        print("Time limit: %.2f seconds" % (time_limit/1000))
    print("Runtime: %.2f seconds \n" % solver.solve_details.time)
    if maximum_gap_allowed > 0:
        print("Gap limit: %.2f" % maximum_gap_allowed)
    print("Gap: %.2f" % result_gap)

display(mip_allocation_output_df) # Jupyter print
# print(mip_allocation_output_df) # Prompt print

# Other CPLEX stats
# mdl.print_information()
# mdl.print_solution()
# mdl.report()
# gap = solver.solve_details.mip_relative_gap

####################################################################################################
####################################################################################################
# 7 - Update L value
l = int(result_makespan)

In [ ]:
# JUST TO EXPORT MODEL AS LP

####################################################################################################
####################################################################################################
# 1 - Inputs / Parameters

print("L (upper bound) value before optimization: %i \n" % l)
l_input = l # to be exported as results

maximum_gap_allowed = 0 # Initialize and reset value
# maximum_gap_allowed = 200  # Maximum GAP allowed

time_limit = 0 # Initialize and reset value
time_limit = 3600000*24 # Time limit in miliseconds

try:
    num_threads = psutil.cpu_count() # Use multi threading / cores
except:
    num_threads = 1

####################################################################################################
####################################################################################################
# 2 - MIP Model
# Model
mdl = Model()

# Variables
cmax = mdl.integer_var(0, int(l), 'cmax') # Makespan used to subtract period costs
xi = [mdl.integer_var(0, int(l), 'xi_%i' % i) for i in range(len(input_tasks))] # xi starting
yij = [[mdl.binary_var('yij_%i_%i' % (i, j)) for j in range(len(input_machines))] for i in range(len(input_tasks))] # yij = 1 if machine j process task i
wik = [[mdl.binary_var('wik_%i_%i' % (i, k)) for k in range(len(input_tasks))] for i in range(len(input_tasks))] 

# Count number of variables
num_var1 = 1 # Cmax
num_var2 = len(xi) # xi
num_var3 = len(yij[0])*len(yij) # yij
num_var4 = len(wik[0])*len(wik) # wik
num_var_total = num_var1 + num_var2 + num_var3 + num_var4 # Total number of variables

# Objective function --> Not considering discount rate yet, only cash flow
mdl.maximize(np.multiply(yij,tasks_profit_matrix).to_numpy().sum() - int(costs_period)*cmax)

# Constraints
num_const1 = 0
num_const2 = 0
num_const3 = 0
num_const4 = 0
num_const5 = 0
num_const6 = 0
num_const7 = 0
num_const8 = 0
num_const9 = 0
num_const10 = 0
num_const_total = 0

print("CONST 1") # sum(yij) <= 1 if j in allowed_machines
for i in range(len(input_tasks)):
    num_const1 += 1
    mdl.add_constraint(sum(yij[i]) <= 1)
print(num_const1, "constraints \n")

print("CONST 2") # sum(yij) = 0 if j NOT in allowed_machines
for i in range(len(input_tasks)):
    num_const2 += 1
    for j in range(len(input_machines)):
        if j not in allowed_machines.iloc[i,0]:
            mdl.add_constraint(yij[i][j] == 0)
print(num_const2, "constraints \n")

print("CONST 3") # xi <= l*sum(yij) --> relation between xi and xij
for i in range(len(input_tasks)):
    num_const3 += 1
    mdl.add_constraint(xi[i] <= l*sum(yij[i])) # l value limits xij
print(num_const3, "constraints \n")

print("CONST 4")  # Hard precedencies --> Qi_hard*sum(yij) <= sum(ykj) --> i only happens if ALL k hard predecessors happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has hard predecessors
        num_const4 += 1
        yij_forall_k = []
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:          
                yij_forall_k.extend(yij[k])
        mdl.add_constraint(Qi_hard.iloc[i,1]*sum(yij[i]) <= sum(yij_forall_k)) # yij = 1 only if all pred processed
print(num_const4, "constraints \n")

print("CONST 5")  # Hard precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_hard.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_hard.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_hard.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const5 += 1
                mdl.add_constraint(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const5, "constraints \n")

print("CONST 6")  # Soft precedencies --> xi + l*(1-sum(yij) >= xk + pkj --> i only happens after k predecessor happens
for i in range(len(input_tasks)):
    if Qi_soft.iloc[i,1] > 0: # Contraint exists only if i has predecessors
        for a in range(len(tasks_pred_soft.columns)-1): # a is a contraint index to loop all predecessors DF --> total num of pred
            k = tasks_pred_soft.iloc[i,a+1] # k is the predecessor IDNUM of i; a+1 because 1st column is IDNUM
            if k!=-1:
                num_const6 += 1
                mdl.add_constraint(xi[i] + l*(1-sum(yij[i])) >= xi[k] + np.sum(np.multiply(np.array(yij[k]),np.array(pij_duration.loc[k]))))
print(num_const6, "constraints \n")

print("CONST 7")  # No-overlapping 1 -->  For 2 tasks i and k: i happens before k OR k before i --> xi + sum(pij*yij) <= xk + l*(1-wik) (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            num_const7 += 1
            mdl.add_constraint(xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))) <= xi[k] + l*(1-wik[i][k]))
print(num_const7, "constraints \n")

print("CONST 8")  # No-overlapping 2 -->  yij + ykj - wik - wki <= 1 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                num_const8 += 1
                mdl.add_constraint(yij[i][j] + yij[k][j] - wik[i][k] - wik[k][i] <= 1)
print(num_const8, "constraints \n")

print("CONST 9")  # No-overlapping 3 -->  yij + ykh + wik + wki <= 2 (k is another task, not a predecessor)
for i in range(len(input_tasks)):
    for k in range(len(input_tasks)):
        if k!=i:
            for j in range(len(input_machines)):
                for h in range(len(input_machines)):
                    if h!=j:
                        num_const9 += 1
                        mdl.add_constraint(yij[i][j] + yij[k][h] + wik[i][k] + wik[k][i] <= 2)
print(num_const9, "constraints \n")

print("CONST 10")  # Define makespan -->  Cmax >= xi + sum(pij*yij)
for i in range(len(input_tasks)):
    num_const10 += 1
    mdl.add_constraint(cmax >= xi[i] + np.sum(np.multiply(np.array(yij[i]),np.array(pij_duration.loc[i]))))
print(num_const10, "constraints \n")

num_const_total = num_const1 + num_const2 + num_const3 + num_const4 + num_const5 + num_const6 + num_const7 # Total number of constraints
print(num_const_total)

# Output filename
outfile = str("CPLEX") + str('-') + str('xi_profit') + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

####################################################################################################
####################################################################################################
# 5 - Export linear model as txt
out_dir = %pwd
mdl.export_as_lp(path = out_dir, basename = outfile)
print("\nMIP model saved as: " + outfile + str('.lp'))
print("\nTotal number of variables: %i" % mdl.number_of_variables)
print("Total number of constraints: %i" % mdl.number_of_constraints)

## 12 - GENETIC ALGORITHM

### 12.1 Definitions and settings

In [15]:
####################################################################################################
####################################################################################################
# 01 - Definitions
# Chromosome --> priority list with tasks indexes that will be allocated. Does not have to include all predecessors
# Phenotype --> chromosome decoded into allocation
# Fitness function --> NPV calculation
# Crossover --> Roulette wheel or Binary tournament
# Mutation --> randomly adds or removes tasks from a list 

####################################################################################################
####################################################################################################
# 02 - GA parameters
def ga_parameters():
    global pop_size, gen_size, mutation_pop_rate, mutation_element_rate, chrom_min_size, chrom_max_size, selective_pressure, chrom_to_search, search_num
    pop_size = 300 # Number of chromossomes for each generation (population size). It has to be an even number for crossover and other operations
    gen_size = 60  # Number of generations
    chrom_min_size = int(len(input_tasks)*0.1) + 1 # Mininmum number of tasks in a chromosome --> Not less than 2
    chrom_max_size = len(input_tasks) # Maximum number of tasks in a chromosome
    mutation_pop_rate = 0.6 # Percentage (%) of the population that will be mutated
    mutation_element_rate = 0.3 # Percentage (%) of the list that will be mutated
    selective_pressure = 0.6 # Percentage of population that is selected (kept due to higher fitness)  
    chrom_to_search = 0.3 # Percentage of last generation that will undergo local search
    search_num = 50 # Number of each search type per individual
    
####################################################################################################
####################################################################################################
# 03 - Aux functions
def create_ga_curr_gen_df(size): # This DF keeps data of a given generation. Input is name of output DF
    ga_curr_gen_df = pd.DataFrame([[0]*8]*size, columns=['Chromosome', 'Chromosome + Hard_pred', 'Start (xi)', 'Duration (pij)', 'Finish (xi)', 'Allocation (machine)', 'Fitness (NPV)', 'Profit']) # Best way to create an empty DF
    return ga_curr_gen_df

### 12.2 Complete chromosome with hard predecessors

In [4]:
####################################################################################################
####################################################################################################
# 04 - Add hard predecessors to input task list

def add_hard_pred(chromossome):
    iterr_input_list = [] # This iterr_input_list check if extra predecessors were added
    input_task_list = chromossome.copy() # This guarantees that input will not be changed
    while input_task_list != iterr_input_list: # This routine will loop until no extra hard predecessors can be added
        iterr_input_list = input_task_list.copy() # This iterr_input_list check if extra hard predecessors were added
        tasks_pred_hard_to_add = tasks_pred_hard[tasks_pred_hard['IDNUM'].isin(input_task_list)] # Filter DF to input task list 

        input_task_priority = dict(zip(input_task_list,range(len(input_task_list)))) # Dict with input taks list and their respective priority position
        tasks_pred_hard_to_add['Priority'] = tasks_pred_hard_to_add['IDNUM'].map(input_task_priority) # Create column to sort
        tasks_pred_hard_to_add.sort_values(['Priority'], inplace = True) # Sort by IDNUM as provided on the input task list

        tasks_pred_hard_to_add.drop(['IDNUM', 'Priority'], axis = 1, inplace = True) # Drop IDNUM and Priority columns --> keep only hard_pred
        tasks_pred_hard_to_add = tasks_pred_hard_to_add.values.flatten(order='C') # Convert to list. 'C' means row major, 'F' would be column major

        input_task_list.extend(tasks_pred_hard_to_add) # Combine hard predecessors with input list
        input_task_list = list(unique_everseen(input_task_list)) # Remove duplicates while keeping order
        input_task_list.remove(-1) # Remove -1 value (-1 = NAN)
    return_list = input_task_list.copy()
    return return_list

### 12.3 Allocation (decode)

In [5]:
####################################################################################################
####################################################################################################
# 05 - Allocation --> Update tasks_available_list (loop through list before going back to beginning)

# Create an input_task_list with all IDNUM values --> This is used only if needed to allocated the whole input_tasks
# input_task_list = tasks_concat['IDNUM'].values.tolist() # List with IDs of tasks that will be allocated

def ga_allocation(list_with_hard_pred): # input list has to have hard precedencies
    input_task_list = list_with_hard_pred.copy() # This guarantees that input will not be changed

    # Initialize machines
    machine_time = input_machines.copy()
    machine_time['Available at time...'] = 0 # Create column to store time of availability of each machine
    machine_time['Processed task duration'] = 0 # Duration of processed task in each machine to decide which machine will process the task
    machine_time['Future time'] = 0 # Future time for each allocation to decide which machine will process the task

    # Create a DF to store cumulative duration for tasks that have predecessors --> also used for machine times
    tasks_cum_duration = tasks_concat.copy()
    tasks_cum_duration.drop(tasks_cum_duration.columns[1:], axis=1, inplace = True) # Keep only IDNUM column
    tasks_cum_duration['Cumulative duration'] = 0 # Create column to store cumulative duration for tasks that have predecessors

    # Create a DF to store allocation
    processed_tasks_allocation = pd.DataFrame(columns=['IDNUM', 'Start (xi)', 'Duration', 'Finish (xi)', 'Machine', 'Profit (xi)'])

    # Initialize DF and list of avaiable tasks (tasks without predecessors)
    tasks_available = tasks_pred.copy() # This is used for the whole loop, being updated with -1 as tasks are allocated
    tasks_available.drop(['DEV TYPE'], axis = 1, inplace = True) # Drop IDNUM column --> keep only 'Pred_' columns

    all_tasks_list = tasks_available.index.values.tolist() # List with IDs of all tasks
    ignored_tasks = [x for x in all_tasks_list if x not in input_task_list] # List with IDs of tasks that will be ignored, not scheduled

    tasks_available = tasks_available[tasks_pred_hard['IDNUM'].isin(input_task_list)] # Filter DF to input task list
    tasks_available = tasks_available.replace(ignored_tasks, -1) # If not on input_task_list, replace by -1 (will be ignored)

    max_num_pred = len(tasks_available.columns) # Maximum number of predecessors that can happen

    tasks_available_list = tasks_available.copy() # This will be a list, different from the DF tasks_available
    tasks_available_list['Count_empty'] = tasks_available.isin([-1]).sum(axis=1) # Column to count empty predecessors
    tasks_available_list = tasks_available_list.loc[tasks_available_list['Count_empty'] == max_num_pred] # Filter tasks that are available (do not have predecessors)
    tasks_available_list = tasks_available_list.index.values.astype(int).tolist() # list with available tasks to be allocated

    input_task_priority = dict(zip(input_task_list,range(len(input_task_list)))) # Dict with input taks list and their respective priority position
    tasks_available_list = sorted(tasks_available_list, key=input_task_priority.get) # Sort tasks_available_list according to input_task_list priority (order)

    # Allocate each task and update available tasks 
    i=0 # Used for processed_tasks_allocation and for printing loop counter
    while len(tasks_available_list) != 0: # Allocate loop while available tasks exists
    # Allocate first task of tasks_available_list
        process_task = tasks_available_list[0] # Select task to be processed (first of the available prioritized)
        process_task_mach_type = tasks_concat.iloc[process_task,1] # Get machine TYPE
        process_task_dr_quantity = tasks_concat.iloc[process_task,2] # Get driving quantity

    # Select machine to process task
        machine_time['Processed task duration'] = process_task_dr_quantity / machine_time['RATE'] # Update processed task duration
        machine_time['Future time'] = np.maximum(machine_time['Available at time...'],tasks_cum_duration.iloc[process_task,1]) + machine_time['Processed task duration'] # Update Future machine time: max between machine available time and task cumulative time (predecessors)

        process_task_allocated_mach = machine_time.loc[machine_time['TYPE'] == process_task_mach_type] # Select machine to allocate --> filter by TYPE
        process_task_allocated_mach_min_time = process_task_allocated_mach['Future time'].min() # Select machine to allocate --> get the minimum future time
        process_task_allocated_mach = process_task_allocated_mach.loc[process_task_allocated_mach['Future time'] == process_task_allocated_mach_min_time] # Get the machines with minimum future time
        process_task_allocated_mach_max_rate = process_task_allocated_mach['RATE'].max() # Maximum rate of available machines
        process_task_allocated_mach = process_task_allocated_mach.loc[process_task_allocated_mach['RATE'] == process_task_allocated_mach_max_rate].index[0] # From the one above, get the highest rate machines. If more than one available get the minimum ID

        process_task_duration = process_task_dr_quantity / input_machines.iloc[process_task_allocated_mach,2] # Get duration based on machine rate
        if process_task_duration != int(process_task_duration): # Check if process_task_duration is integer --> It has to be for MIP solvers
            process_task_duration = int(process_task_duration) + 1 # Round up if not integer
        process_task_cum_duration = process_task_duration + tasks_cum_duration.iloc[process_task,1] # Update cumulative duration of task predecessors with duration of task

        machine_time.iloc[process_task_allocated_mach,3] += process_task_duration # Update machine time with processed task duration
        machine_time.iloc[process_task_allocated_mach,3] = max(machine_time.iloc[process_task_allocated_mach,3], process_task_cum_duration) # Update machine time with maximum of (updated machine time, cumulative duration)

    # Update processed task time (end) to 'Cumulative duration' of its sucessors
        process_task_succ = tasks_succ.loc[tasks_succ['IDNUM'] == process_task,tasks_succ.columns[1:]].values # Get successors of processed task
        process_task_succ = process_task_succ[process_task_succ!=-1].tolist() # Ignore -1 , since it represents NAN    
        if len(process_task_succ)> 0:
            for succ in process_task_succ:
                tasks_cum_duration.iloc[succ,1] = machine_time.iloc[process_task_allocated_mach,3] # Get value of finish of precessed task

    # Update allocation DF --> processed_tasks_allocation 
        processed_tasks_allocation.loc[i] = process_task # Updated list of processed tasks --> IDNUM. LOC to include row
        processed_tasks_allocation.iloc[i,1] = machine_time.iloc[process_task_allocated_mach,3] - process_task_duration # Updated list of processed tasks --> Task start (xi)
        processed_tasks_allocation.iloc[i,2] = process_task_duration # Updated list of processed tasks --> Duration
        processed_tasks_allocation.iloc[i,3] = processed_tasks_allocation.iloc[i,1] + processed_tasks_allocation.iloc[i,2] # Updated list of processed tasks --> Finish
        processed_tasks_allocation.iloc[i,4] = process_task_allocated_mach # Updated list of processed tasks --> Allocated machine
        processed_tasks_allocation.iloc[i,5] = tasks_profit.iloc[processed_tasks_allocation.iloc[i,0],0] # Updated list of processed tasks --> Profit
        i += 1
        
    # Remove processed task from tasks_available DF
        tasks_available = tasks_available.drop(process_task) # Drop the row of the processed task
        tasks_available = tasks_available.replace(process_task, -1) # Replace processed task by -1 in tasks_available 'Pred_' coluns

    # Get new tasks_available_list from the updated tasks_available DF and append to previous list
        tasks_available_list.remove(process_task) # Remove processed task from available list
        prev_tasks_available_list = tasks_available_list.copy() # Previous list is kept at higher priority, thus list is not reset

        tasks_available_list = tasks_available.copy() # This will be a list, different from the DF tasks_available
        tasks_available_list['Count_empty'] = tasks_available.isin([-1]).sum(axis=1) # Column to count empty predecessors
        tasks_available_list = tasks_available_list.loc[tasks_available_list['Count_empty'] == max_num_pred] # Filter tasks that are available (do not have predecessors)
        tasks_available_list = tasks_available_list.index.values.astype(int).tolist() # list with available tasks to be allocated
        tasks_available_list = sorted(tasks_available_list, key=input_task_priority.get) # Sort tasks_available_list according to input_task_list priority (order)

        prev_tasks_available_list.extend(tasks_available_list) # Add Combine hard predecessors with input list
        tasks_available_list = prev_tasks_available_list.copy() # Rename extended list
        tasks_available_list = list(unique_everseen(tasks_available_list)) # Remove duplicates while keeping order

    # Remove processed task from the new tasks_available_list
        processed_tasks_list = processed_tasks_allocation['IDNUM'].values.tolist() # Add processed task to processed_tasks_list
        tasks_available_list = [x for x in tasks_available_list if x not in processed_tasks_list] # Get the updated tasks_available_list and remove the processed tasks

    # Get makespan (L)
    l = machine_time['Available at time...'].max() # Upper-bound for time index --> i.e. makespan
    if l != int(l): # Check if L is integer --> It has to be for MIP solvers
        l = int(l) + 1 # Round up if not integer

    # Calculate NPV for an allocation (processed_tasks_allocation) --> GA Fitness function
    npv = 0
    for i in range(len(processed_tasks_allocation)):
        npv += tasks_profit.iloc[processed_tasks_allocation.iloc[i,0],0]/((1+day_disc_rate)**processed_tasks_allocation.iloc[i,1])
    npv += - l*costs_period # Period costs multiplied by L subtracting NPV
    
    # Sort DF processed_tasks_allocation according to input list (Chromosome order)
    processed_tasks_allocation['Chrom_order'] = processed_tasks_allocation['IDNUM'].map(input_task_priority) # Add a column to DF to sort
    processed_tasks_allocation.sort_values(['Chrom_order'], inplace = True)

    # Generate lists to return to ga_curr_gen_df 
    start_xi = processed_tasks_allocation['Start (xi)'].values.tolist()
    duration_pij = processed_tasks_allocation['Duration'].values.tolist()
    finish_xi = processed_tasks_allocation['Finish (xi)'].values.tolist()
    machine_j = processed_tasks_allocation['Machine'].values.tolist()
    profit_xi = processed_tasks_allocation['Profit (xi)'].values.tolist()
    
    return start_xi, duration_pij, finish_xi, machine_j, npv, profit_xi

### 12.4 Initialize population

In [6]:
####################################################################################################
####################################################################################################
# 06 - Initialization
ga_parameters()
def ga_init_chrom(x): # Create an initialization chromossome (list)
    input_tasks_list = range(len(input_tasks)) # All tasks possible (input)
    chrom_size = randint(chrom_min_size, chrom_max_size) # Chromosome (list) size
    chrom_init = random.sample(input_tasks_list, chrom_size) # Sample of input_tasks_list
    return chrom_init

def ga_init(): # Create a population to initialize
    init = create_ga_curr_gen_df(pop_size) # DF to keep data of initialization 
    init['Chromosome'] = init['Chromosome'].swifter.allow_dask_on_strings(enable=True).apply(ga_init_chrom) # Function to generate 1st column: Chromosome
    init['Chromosome + Hard_pred'] = init['Chromosome'].swifter.allow_dask_on_strings(enable=True).apply(add_hard_pred) # Function to generate 2n column: Chromosome with hard predecessors
    init['Start (xi)'], init['Duration (pij)'], init['Finish (xi)'], init['Allocation (machine)'], init['Fitness (NPV)'], init['Profit'] = zip(*init['Chromosome + Hard_pred'].swifter.allow_dask_on_strings(enable=True).apply(ga_allocation)) # Function to calculate other columns
    return init

# df['p1'], df['p2'], df['p3'], df['p4'], df['p5'], df['p6'] = zip(*df['num'].map(powers)) # Example of an apply function that return multiple results / columns
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1) # Example of apply function that requires multiple inputs form pandas DF

### 12.5 Crossover

In [7]:
####################################################################################################
####################################################################################################
# 07 - Crossover - Roulette wheel 
def ga_crossover_roulette(df_series, npv_list): # Input example: gen_0.iloc[:,0], gen_0.iloc[:,6]  --> column 0 and 6 of df gen_0
    wheel_percent = npv_list / npv_list.sum() # Create NPV percent list
    wheel_percent_cumulative = wheel_percent.cumsum() # Create NPV cumulative percent list
    wheel_dict = dict(zip(wheel_percent_cumulative,df_series)) # Dict zip percent and chromosomes
    
    offspring = []
    for i in range(int(pop_size/2)):
    # Roulette
        parent1_beg, parent1_fin, parent2_beg, parent2_fin = [], [], [], []
        while (parent1_beg == parent2_beg) and (parent1_fin == parent2_fin): # Guarantee parent1 != parent2
            parent1 = wheel_dict[min(filter(lambda x: x >= random.uniform(0, 1), wheel_percent_cumulative))] # Select parent based on a random percentage
            parent2 = wheel_dict[min(filter(lambda x: x >= random.uniform(0, 1), wheel_percent_cumulative))] # Select parent based on a random percentage 
            
            parent1_beg = parent1[:(int(len(parent1)/2))] # Take the first half of the parent1
            parent1_fin = [x for x in parent1 if x not in parent1_beg] # Take the second half of parent1
            parent2_beg = parent2[:(int(len(parent2)/2))] # Take the first half of the parent2
            parent2_fin = [x for x in parent2 if x not in parent2_beg] # Take the second half of parent2
        
        parent1_beg.extend(parent2_fin) # Crossover parent1 x parent2
        parent2_beg.extend(parent1_fin) # Crossover parent2 x parent1
        
        offspring1 = list(unique_everseen(parent1_beg)) # Remove duplicated values in list, if exists
        offspring2 = list(unique_everseen(parent2_beg)) # Remove duplicated values in list, if exists
        
        offspring.append(offspring1) # Add each crossover to return list
        offspring.append(offspring2) # Add each crossover to return list
    return offspring # Input list is not returned, only offspring

In [8]:
####################################################################################################
####################################################################################################
# 07 - Crossover - Binary tournament
def ga_crossover_bin_tournament(df_series, npv_list): # Input example: gen_0.iloc[:,0], gen_0.iloc[:,6]  --> column 0 and 6 of df gen_0   
    offspring = []
    for i in range(int(pop_size/2)):
        j, k, l, m = random.sample(range(pop_size), 4) # Random indexes to selec parents
        
    # Binary tournament      
        parent1_index = [index for index, x in enumerate(npv_list) if x == max(npv_list[j],npv_list[k]) and index in [j,k]][0] # Get [0] value (they can be equal) of the list of indexes which have x has max value
        parent1 = df_series.get(parent1_index) # Get parent1, the one with best NPV from j and k
        
        parent2_index = [index for index, x in enumerate(npv_list) if x == max(npv_list[l],npv_list[m]) and index in [l,m]][0] # Get [0] value (they can be equal) of the list of indexes which have x has max value
        parent2 = df_series.get(parent2_index) # Get parent2, the one with best NPV from l and m
    
    # Crossover
        parent1_beg = parent1[:(int(len(parent1)/2))] # Take the first half of the parent1
        parent1_fin = [x for x in parent1 if x not in parent1_beg] # Take the second half of parent1
        parent2_beg = parent2[:(int(len(parent2)/2))] # Take the first half of the parent2
        parent2_fin = [x for x in parent2 if x not in parent2_beg] # Take the second half of parent2
        
        parent1_beg.extend(parent2_fin) # Crossover parent1 x parent2
        parent2_beg.extend(parent1_fin) # Crossover parent2 x parent1
        
        offspring1 = list(unique_everseen(parent1_beg)) # Remove duplicated values in list, if exists
        offspring2 = list(unique_everseen(parent2_beg)) # Remove duplicated values in list, if exists
        
        offspring.append(offspring1) # Add each crossover to return list
        offspring.append(offspring2) # Add each crossover to return list
    return offspring # Input list is not returned, only offspring

### 12.6 Mutation

In [9]:
####################################################################################################
####################################################################################################
# 08 - Mutation (add or remove 'mutation_element_rate' itens of "mutation_pop_rate" lists of the input)
def ga_mutation(crossover): # Result of a ga_crossover(df_series) function is this input    
    mutation_chrom_number = int(len(crossover)*mutation_pop_rate) # number of lists that will be mutated
    for i in range(mutation_chrom_number):
        j = randint(0, len(crossover)-1) # Decide which iten will be mutated. It can repeat
        add_remove_decision = randint(1, 10) # Decide randomly if mutation will add or remove itens
        
        if add_remove_decision > 5: # Mutation add itens
            mutation_element_number = int(len(crossover[j])*mutation_element_rate)# Number of itens that will be added in original list
            
            not_in_list = [x for x in range(len(input_tasks)) if x not in crossover[j]] # Elements not in input chromosome
            list_to_add = random.sample(not_in_list, min(mutation_element_number,len(not_in_list))) # Get elements not_in_list randomly
            
            if len(list_to_add) > 0:
                insert_position = randint(0, len(crossover[j])-1) # Randomly, get position that will receive added itens
                crossover[j][insert_position:insert_position] = list_to_add # Insert list_to_add elements in a random place
        
        else: # Mutation remove itens
            mutation_element_number = int(len(crossover[j])*mutation_element_rate) # Number of itens that will be removed in original list
            list_to_remove = random.sample(crossover[j], mutation_element_number) # Get, randomly, elements to remove 
            crossover[j] = [x for x in crossover[j] if x not in list_to_remove] # List with elements removed
    return crossover # Changes in-place

### 12.7 Update generation DF (gen i)

In [10]:
####################################################################################################
####################################################################################################
# 09 - Process updated population
def ga_update_gen(population_input): # Input is a population list, e.g. result from mutation (any size, pop_size not required)
    curr_gen = create_ga_curr_gen_df(len(population_input)) # DF to keep data of generation
    curr_gen['Chromosome'] = population_input # Input list is the chromosome column
    curr_gen['Chromosome + Hard_pred'] = curr_gen['Chromosome'].swifter.allow_dask_on_strings(enable=True).apply(add_hard_pred) # Function to generate 2n column: Chromosome with hard predecessors
    curr_gen['Start (xi)'], curr_gen['Duration (pij)'], curr_gen['Finish (xi)'], curr_gen['Allocation (machine)'], curr_gen['Fitness (NPV)'], curr_gen['Profit'] = zip(*curr_gen['Chromosome + Hard_pred'].swifter.allow_dask_on_strings(enable=True).apply(ga_allocation)) # Function to calculate other columns
       
    return curr_gen # Gen_i DF for gen_i offsprings

### 12.8 Selection

In [11]:
####################################################################################################
####################################################################################################
# 10 - Select the next gen population from current gen and their offspring
def ga_selection(gen_i, gen_i_offspring): # Input is current generation and their offspring --> 2 gen_i DFs
    next_gen = pd.concat([gen_i,gen_i_offspring]).copy() # Concat previous gen to their offspring     
    
    next_gen.sort_values(['Fitness (NPV)'], ascending = False, inplace = True) # Sort by Fitness (NPV)
    next_gen = next_gen.reset_index(drop=True) # Reset index
    
    keep_indexes = int(pop_size*selective_pressure) # Keep first (selective_pressure)% of pop_size and complete pop_size randomly
    drop_indexes_list = range(keep_indexes + 1, len(next_gen)) # This is the list of possible indexes, i.e. the ones outside the best (selective_pressure)%
    drop_indexes = random.sample(drop_indexes_list, len(next_gen) - pop_size) # Indexes to be dropped. Some are kept to complete the pop_size
    
    next_gen = next_gen.drop(drop_indexes) # Drop the extra chromosomes, keeping only pop_size on the DF
    next_gen.reset_index(drop = True, inplace = True) # Reset index
   
    return next_gen # After selection of parents + offspring population

### 12.9 Outputs

In [12]:
####################################################################################################
####################################################################################################
# 11 - Create and export output stats and allocation (sheet1 = stats, sheet2 = allocation) and print results
def ga_output(gen_df_row, npv_stats_df, runtime): # Input is a generation df row, npv stats (min, mean, max) and runtime.
    tasks = gen_df_row[1].copy() # Copy task list from the best chromosome (higher NPV), i.e. input
    tasks.extend(range(len(input_tasks))) # Complete task list (Chromosome + Hard_pred) with tasks that were not scheduled
    tasks = list(unique_everseen(tasks)) # Remove duplicates
             
    # Create DFs             
    ga_stats_output = pd.DataFrame([[0]*21]*1, columns=['Input file', 'Input tasks', 'Input machines', 'Input L (days)', 'Solver', 'Algorithm', 'Variables number', 'Constraints number', 'Time limit (hour)', 'Walltime (hour)', 'Optimized value', 'Best bound', 'GAP', 'NPV ($)', 'Makespan (days)', 'pop_size', 'gen_size', 'mutation_pop_rate', 'mutation_element_rate', 'chrom_min_size', 'selective_pressure']) # Best way to create an empty DF
    ga_allocation_output = pd.DataFrame([[0]*6]*len(tasks), columns=['Task', 'Start', 'Finish', 'Machine', 'Duration (pij)', 'Profit']) # Best way to create an empty DF --> size of chromosome + hard_pred

    # Populate allocation
    ga_allocation_output['Task'] = tasks # Task IDNUM --> Chromosome + Hard_pred
    ga_allocation_output['Start'] = pd.Series(gen_df_row[2]) # Start
    ga_allocation_output['Finish'] = pd.Series(gen_df_row[4]) # Finish
    ga_allocation_output['Machine'] = pd.Series(gen_df_row[5]) # Machine
    ga_allocation_output['Duration (pij)'] = pd.Series(gen_df_row[3]) # Duration (pij)
    ga_allocation_output['Profit'] = pd.Series(gen_df_row[7]) # Profit
    
    # Populate stats
    ga_stats_output.iloc[0,0] = inputfile[0] # Input xlsx file used
    ga_stats_output.iloc[0,1] = len(input_tasks) # Number of tasks
    ga_stats_output.iloc[0,2] = len(input_machines) # Number of machines
    ga_stats_output.iloc[0,3] = "NA" # l value used as input
    ga_stats_output.iloc[0,4] = "Genetic Algorithm" # Solver: CBC; CP-SAT; CPLEX
    ga_stats_output.iloc[0,5] = "GA" # Model: xi_makespan; xijt; xi_profit
    ga_stats_output.iloc[0,6] = "NA"
    ga_stats_output.iloc[0,7] = "NA"
    ga_stats_output.iloc[0,8] = "NA" # Time limit in hours
    ga_stats_output.iloc[0,9] = runtime/3600 # Wall time in hours
    ga_stats_output.iloc[0,10] = gen_df_row[6] # Optimized value
    ga_stats_output.iloc[0,11] = "NA" # Best bound
    ga_stats_output.iloc[0,12] = "NA" # GAP
    ga_stats_output.iloc[0,13] = gen_df_row[6] # NPV
    ga_stats_output.iloc[0,14] = pd.to_numeric(ga_allocation_output['Finish']).max() # Makespan
    ga_stats_output.iloc[0,15] = pop_size # GA parameter - Number of chromosomes per generation
    ga_stats_output.iloc[0,16] = gen_size # GA parameter - Number of generations
    ga_stats_output.iloc[0,17] = mutation_pop_rate # GA parameter - Percentage (%) of the population that will be mutated
    ga_stats_output.iloc[0,18] = mutation_element_rate # GA parameter - Percentage (%) of the list that will be mutated
    ga_stats_output.iloc[0,19] = chrom_min_size # GA parameter - Mininmum number of tasks in a chromosome
    ga_stats_output.iloc[0,20] = selective_pressure # GA parameter - Selective pressure

    # Sort ga_allocation_output by IDNUM
    ga_allocation_output.sort_values(['Task'], inplace = True) # Sort by IDNUM
    ga_allocation_output = ga_allocation_output.reset_index(drop=True) # Reset index
    
    # Convert allocation output DF to int (so as to have number in Excel)
    ga_allocation_output_xlsx = ga_allocation_output.copy()
    ga_allocation_output_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
    ga_allocation_output_xlsx = ga_allocation_output_xlsx.applymap(str) # Convert object to str
    ga_allocation_output_xlsx = ga_allocation_output_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer
    
    # Create output DFs as sheets
    sh1 = pd.DataFrame(ga_stats_output)# Sheet 1 - Stats
    sh2 = pd.DataFrame(ga_allocation_output_xlsx) # Sheet 2 - Allocation
    sh3 = pd.DataFrame(npv_stats_df) # Sheet 3 - NPV stats

    # Output filename
    outfile = str(ga_stats_output.iloc[0,5]) + str('-') + str(len(input_tasks)) + str('_tasks-') + str(len(input_machines)) + str('_mach-') + time.strftime("%Y_%m_%d-%H%M%S") # Solver + Algorithm + # tasks + # machines + Datetime

    # Export output in Excel file
    writer = pd.ExcelWriter(outfile + str('.xlsx'), engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
    sh1_name = 'STATS' # Sheet names
    sh2_name = 'ALLOCATION' # Sheet names
    sh3_name = 'NPV stats' # Sheet names

    sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object
    sh2.to_excel(writer, sheet_name=sh2_name) # Convert the dataframe to an XlsxWriter Excel object
    sh3.to_excel(writer, sheet_name=sh3_name) # Convert the dataframe to an XlsxWriter Excel object

    # Close the Pandas Excel writer and output the Excel file
    writer.save()
    print("\nResults saved as: " + outfile + str('.xlsx'))

    ####################################################################################################
    # Plotly export (image)
    ga_allocation_output_xlsx['Machine'] = ga_allocation_output_xlsx['Machine'].astype(str) # Change machine to string to have proper legend
    fig = ff.create_gantt(ga_allocation_output_xlsx, index_col='Machine',show_colorbar=True, group_tasks=True) # Create Plotly gantt graph

    # Get x axis with linear format from 0 to L*110%
    fig['layout']['xaxis']['tickformat'] = '%L'
    fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
    fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
    fig.layout.xaxis.type = 'linear'
    fig.layout.legend.traceorder = 'normal'

    fig.write_image(outfile + str('.png'))# Save as image
    print("\nImage saved as: " + outfile + str('.png'))

    ####################################################################################################
    # Print results
    print("\n>>>>>>>>>> RESULTS <<<<<<<<<<\n")
    print("Optimized value (NPV): {:,.0f}".format(ga_stats_output.iloc[0,13]).replace(',', ' ')) # NPV
    print("\nMakespan: %i" % ga_stats_output.iloc[0,14]) # Makespan
    print("Runtime: %.2f seconds \n" % runtime) # runtim

    display(ga_allocation_output) # Jupyter print
    # print(ga_allocation_output) # Prompt print

### 12.10 Local search

In [13]:
####################################################################################################
####################################################################################################
# 12 - Local search to improve GA results
def ga_local_search(gen_i_chromosomes): # Input is a generation chromosome list, sorted by fitness value
    gen_i_chromosomes = gen_i_chromosomes.values.tolist() # Required to change Series type to list type
    input_plus_search = gen_i_chromosomes.copy() # Copy to not change input
    search_chrom_number = int(len(gen_i_chromosomes)*chrom_to_search) + 1 # number of chromosomes that will be searched
    search_chrom_number = min(search_chrom_number, len(gen_i_chromosomes))

    for i in range(search_chrom_number): # Loop through chromosomes that will be improved
        # Addition PROFIT rank
        add_profit_df = tasks_profit.drop(gen_i_chromosomes[i]) # drop tasks already in chromosome
        add_profit_df = add_profit_df[add_profit_df['PROFIT'] > 0] # drop also negative PROFIT
        add_profit_df.sort_values(['PROFIT'], ascending = False, inplace = True) # Sort by PROFIT (most positive first)

        # Removal PROFIT rank
        rem_profit_df = tasks_profit[tasks_profit.index.isin(gen_i_chromosomes[i])] # Filtered PROFIT DF (only chromosome tasks)
        rem_profit_df.sort_values(['PROFIT'], ascending = True, inplace = True) # Sort by PROFIT (most negative first)

        number_elements_add = len(add_profit_df) // (min(search_num, len(add_profit_df)) + 1) # Number of tasks to add per run
        number_elements_remove = len(rem_profit_df) // (min(search_num, len(rem_profit_df)) + 1) # Number of tasks to remove --> one-by-one or in groups

        # SEARCH 1 and SEARCH 3 (ADD / ADD & REMOVE)
        for j in range(min(search_num, len(add_profit_df))): #Number of search runs per chromosome
        
        # search 1 --> All positive PROFIT elements will be added (one-by-one or in groups)
            list_to_add = add_profit_df.index[j:(j+number_elements_add)] # Get elements/tasks to add
            if len(list_to_add) > 0:
                search1 = gen_i_chromosomes[i].copy()
                insert_position = randint(0, len(gen_i_chromosomes[i])-1) # Randomly, get position that will receive added itens
                search1[insert_position:insert_position] = list_to_add # Insert list_to_add elements in a random place
                input_plus_search.append(search1) # Store search 1 result

        # search 2 (REMOVE)
            list_to_remove = rem_profit_df.index[j:(j+number_elements_add)] # Get elements/tasks to remove
            search2 = [x for x in gen_i_chromosomes[i] if x not in list_to_remove] # Search 2 chromosome
            input_plus_search.append(search2) # Store search 2 result

        # search 3 --> Remove negative tasks from search 1 result
            if len(list_to_add) > 0:
                list_to_remove = rem_profit_df.index[j:(j+number_elements_add)] # Get elements/tasks to remove
                search3 = [x for x in search1 if x not in list_to_remove] # Search 1 with elements removed = Search 3
                input_plus_search.append(search3) # Store search 3 result
                
    return input_plus_search # list of neighborhood chromosomes + input chromosomes

### Main

In [16]:
####################################################################################################
####################################################################################################
# 13 - Main function of GA that solves the UG scheduling problem
start_time = process_time() # Get start time to calculate runtime
ga_parameters() # Load main parameters
gen_0 = ga_init() # Initialize population

# Get NPV stats from all gen
npv_stats_df = pd.DataFrame([[0]*4]*(gen_size + 2), columns=['Gen', 'NPV min', 'NPV mean', 'NPV max']) # Best way to create an empty DF
npv_stats_df.iloc[0,1] = gen_0['Fitness (NPV)'].min() # Get gen NPV stats --> Min
npv_stats_df.iloc[0,2] = gen_0['Fitness (NPV)'].mean() # Get gen NPV stats --> Mean
npv_stats_df.iloc[0,3] = gen_0['Fitness (NPV)'].max() # Get gen NPV stats --> Max
    
for i in range(gen_size): # One loop for each generation
    # Crossover options --> ga_crossover_roulette OR ga_crossover_bin_tournament --> SAME INPUTS
    vars()['gen_' + str(i) + '_crossover'] = ga_crossover_bin_tournament(vars()['gen_' + str(i)].iloc[:,0], vars()['gen_' + str(i)].iloc[:,6]) # Crossover on current gen
    vars()['gen_' + str(i) + '_mutation'] = ga_mutation(vars()['gen_' + str(i) + '_crossover']) # Mutation on offspring crossover
    vars()['gen_' + str(i) + '_offspring'] = ga_update_gen(vars()['gen_' + str(i) + '_mutation']) # Update mutated list to gen DF
    vars()['gen_' + str(i+1)] = ga_selection(vars()['gen_' + str(i)], vars()['gen_' + str(i) + '_offspring']) # Select between gen_i and gen_i_offspring

    # NPV stats
    npv_stats_df.iloc[(i+1),0] = (i + 1) # Generation number
    npv_stats_df.iloc[(i+1),1] = vars()['gen_' + str(i+1)]['Fitness (NPV)'].min() # Get gen NPV stats --> Min
    npv_stats_df.iloc[(i+1),2] = vars()['gen_' + str(i+1)]['Fitness (NPV)'].mean() # Get gen NPV stats --> Mean
    npv_stats_df.iloc[(i+1),3] = vars()['gen_' + str(i+1)]['Fitness (NPV)'].max() # Get gen NPV stats --> Max   
    
    print('gen_' + str(i+1) + ' calculated.')

# Sort last gen_i DF by NPV
vars()['gen_' + str(i+1)].sort_values(['Fitness (NPV)'], ascending = False, inplace = True) # Sort by Fitness (NPV)
vars()['gen_' + str(i+1)] = vars()['gen_' + str(i+1)].reset_index(drop=True) # Reset index

# Local search
vars()['gen_' + str(i+1) + '_loc_search'] = ga_local_search(vars()['gen_' + str(i+1)].iloc[:,0]) # Local search of last GA generation
vars()['gen_' + str(i+1) + '_loc_search_results'] = ga_update_gen(vars()['gen_' + str(i+1) + '_loc_search']) # Update local search list
vars()['gen_' + str(i+2)] = ga_selection(vars()['gen_' + str(i)], vars()['gen_' + str(i) + '_offspring']) # Select between gen_i and gen_i_offspring
# NPV stats
npv_stats_df.iloc[(i+2),0] = (i + 2) # Generation number
npv_stats_df.iloc[(i+2),1] = vars()['gen_' + str(i+2)]['Fitness (NPV)'].min() # Get gen NPV stats --> Min
npv_stats_df.iloc[(i+2),2] = vars()['gen_' + str(i+2)]['Fitness (NPV)'].mean() # Get gen NPV stats --> Mean
npv_stats_df.iloc[(i+2),3] = vars()['gen_' + str(i+2)]['Fitness (NPV)'].max() # Get gen NPV stats --> Max   

print('gen_' + str(i+2) + ' (local search) calculated.')

runtime = process_time() - start_time
ga_output(vars()['gen_' + str(i+2)].iloc[0,:], npv_stats_df, runtime) # Output first row: best NPV

C:\Users\alexa\AppData\Roaming\Python\Python37\site-packages\tqdm\std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version




gen_1 calculated.



gen_2 calculated.



gen_3 calculated.



gen_4 calculated.


...


gen_60 calculated.



gen_61 (local search) calculated.

Results saved as: GA-2044_tasks-5_mach-2020_02_11-221239.xlsx


C:\ProgramData\Anaconda3\lib\site-packages\plotly\figure_factory\_gantt.py:49: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated




Image saved as: GA-2044_tasks-5_mach-2020_02_11-221239.png

>>>>>>>>>> RESULTS <<<<<<<<<<

Optimized value (NPV): 35 901 468

Makespan: 2757
Runtime: 100459.98 seconds 



,Task,Start,Finish,Machine,Duration (pij),Profit
0,0,251.0,261.0,2.0,10.0,-43863.0
1,1,564.0,575.0,1.0,11.0,-32319.0
2,2,1125.0,1140.0,0.0,15.0,-33036.0
3,3,1528.0,1536.0,2.0,8.0,-31596.0
4,4,1856.0,1864.0,2.0,8.0,-31596.0
...,...,...,...,...,...,...
2039,2039,2520.0,2524.0,3.0,4.0,45689.0
2040,2040,2517.0,2520.0,3.0,3.0,45946.0
2041,2041,2509.0,2512.0,3.0,3.0,40172.0
2042,2042,2504.0,2505.0,4.0,1.0,18278.0


In [17]:
####################################################################################################
####################################################################################################
# 14 - Print all Fitness Values (NPV) per generation
npv_gen_df = pd.DataFrame(gen_0['Fitness (NPV)'])
for i in range(gen_size + 1):
    df_to_append = pd.DataFrame(vars()['gen_' + str(i + 1)]['Fitness (NPV)']) 
    npv_gen_df = pd.concat([npv_gen_df, df_to_append], axis = 1, ignore_index=True)
print(len(np.unique(npv_gen_df.values.flatten())), 'unique values')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
npv_gen_df

6095 unique values


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
0,7.510418e+06,3.553425e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.590147e+07,3.590147e+07,3.590147e+07,3.590147e+07,3.590147e+07,3.590147e+07
1,3.361342e+07,3.533615e+07,3.533615e+07,3.533615e+07,3.534897e+07,3.534897e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.570314e+07,3.570314e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07,3.576815e+07
2,3.593330e+06,3.522678e+07,3.522678e+07,3.525962e+07,3.533615e+07,3.533615e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.562075e+07,3.562075e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.565891e+07,3.568478e+07,3.568478e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07,3.571292e+07
3,1.105138e+07,3.519343e+07,3.519343e+07,3.522678e+07,3.525962e+07,3.525962e+07,3.534897e+07,3.534897e+07,3.535169e+07,3.544073e+07,3.552600e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.553425e+07,3.561924e+07,3.561924e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.562075e+07,3.567771e+07,3.567771e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.568478e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.570314e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07,3.571210e+07
4,2.090621e+07,3.512670e+07,3.512670e+07,3.519343e+07,3.522678e+07,3.522678e+07,3.533615e+07,3.533615e+07,3.534897e+07,3.540540e+07,3.544073e+07,3.552600e+07,3.552600e+07,3.552600e+07,3.552600e+07,3.553425e+07,3.553425e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.561924e+07,3.565891e+07,3.565891e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.567771e+07,3.56

## 13 - PLOTLY RESULTS (ANY SOLVER)

In [ ]:
# Decide what to plot
plot_input = 'MIP-solver' # Results from CBC, CP-SAT or CPLEX
plot_input = 'GA allocations' # processed_tasks_allocation --> Comment to show MIP-solver results

####################################################################################################
####################################################################################################
# Get plotly_df 
if plot_input == 'GA allocations': # Plotly DF for l (time upper-bound) allocation --> processed_tasks_allocation
    plotly_df = pd.DataFrame({'Task':processed_tasks_allocation['IDNUM'],'Start':processed_tasks_allocation['Task start (xi)'],'Finish':(processed_tasks_allocation['Task start (xi)'] + processed_tasks_allocation['Duration']),'Machine':processed_tasks_allocation['Machine']})
    plotly_df['Machine'] = plotly_df['Machine'].astype(str) # Change machine to string to have proper legend
    plotly_df.reset_index(drop=True, inplace=True) # Reset index
else:
    plotly_df = mip_allocation_output_df # Plotly for MIP Solver output
    plotly_df['Machine'] = plotly_df['Machine'].astype(str) # Change machine to string to have proper legend

# Create Plotly gantt graph
fig = ff.create_gantt(plotly_df, index_col='Machine',show_colorbar=True, group_tasks=True)

# Get x axis with linear format from 0 to L*110%
fig['layout']['xaxis']['tickformat'] = '%L'
fig['layout']['xaxis']['tickvals'] = np.arange(0,l*1.1)
fig['layout']['xaxis']['ticktext'] = list(range(len(fig['layout']['xaxis']['tickvals'])))
fig.layout.xaxis.type = 'linear'
fig.layout.legend.traceorder = 'normal'

# Show Gantt and DF with table
fig.show()

# Show allocation DF
if plot_input == 'GA allocations':
    display(plotly_df) # Jupyter print --> Greedy
else:
    display(mip_output_df) # Jupyter print --> MIP

# Save as image
fig_name = 'fig_' + time.strftime("%Y_%m_%d-%H%M") + '.png' # Get sheet name with current time
fig.write_image(fig_name)

## 14 - EXPORT RESULTS TO XLSX (ANY SOLVER)

In [ ]:
# Decide what to plot
plot_input = 'MIP-solver' # Results from CBC, CP-SAT or CPLEX
plot_input = 'GA allocations' # processed_tasks_allocation --> Comment to show MIP-solver results

####################################################################################################
####################################################################################################
# Convert output DF to int (so as to have number in Excel)
if plot_input == 'GA allocations': # Plotly DF for l (time upper-bound) allocation --> processed_tasks_allocation
    plotly_df = pd.DataFrame({'Task':processed_tasks_allocation['IDNUM'],'Start':processed_tasks_allocation['Task start (xi)'],'Finish':(processed_tasks_allocation['Task start (xi)'] + processed_tasks_allocation['Duration']),'Machine':processed_tasks_allocation['Machine']})
    plotly_df['Machine'] = plotly_df['Machine'].astype(str) # Change machine to string to have proper legend
    plotly_df.reset_index(drop=True, inplace=True) # Reset index
    mip_output_df_xlsx = plotly_df.copy()
else:
    mip_output_df_xlsx = mip_allocation_output_df.copy()

mip_output_df_xlsx.fillna(-1,inplace=True) # Replace NAN by -1
mip_output_df_xlsx = mip_output_df_xlsx.applymap(str) # Convert object to str
mip_output_df_xlsx = mip_output_df_xlsx.apply(pd.to_numeric, errors='raise', downcast='integer').fillna(-1) # Convert str to integer

# Create output DFs as sheets
sh1 = pd.DataFrame(mip_output_df_xlsx) # sheet 1

# TK to get output filename and directory
root = Tk() # Create Tk root
root.withdraw() # Hide the main window
root.call('wm', 'attributes', '.', '-topmost', True)
outfile = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=(("Excel Spreadsheet", "*.xlsx"),("All Files", "*.*") ))
%gui tk

# Export output in Excel file
writer = pd.ExcelWriter(outfile, engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine
sh1_name = 'Solved in' + time.strftime("%Y_%m_%d-%H%M") # Get sheet name with current time
sh1.to_excel(writer, sheet_name=sh1_name) # Convert the dataframe to an XlsxWriter Excel object

# Close the Pandas Excel writer and output the Excel file
writer.save()